# SQL for Data Analysis  
### Introductory Workshop
*D‑Lab, UC Berkeley*

### Welcome & Environment Check 

Estimated time: 8 minutes

## Table of Contents  
1. [Why SQL?](#why-sql)
3. [Relational Databases](#relational)    
4. [Importing CSV → SQLite](#sqlite)  
5. [SQLite Data Types & NULLs](#types)  
6. [SELECT & Derived Columns](#select)  
7. [Filtering Rows with WHERE](#where)  
8. [Aggregates & GROUP BY](#groupby)  
9. [Sorting & Paging Results](#orderby)   
10. [Key Points & Next Steps](#keypoints)

<div class="alert alert-success"> 
<b>Learning Goals</b><br><br>
By the end of this workshop you will be able to:
<ul>
<li>Understand why one would use SQL and why and how it is complementary to Pandas</li>
<li>Write basic queries with <code>SELECT</code>, create aliases, and build derived columns.</li>
<li>Filter rows using <code>WHERE</code>, sort and paginate result sets with <code>ORDER BY</code>, <code>LIMIT</code>, <code>OFFSET</code>.</li>
<li>Summarise data with aggregates &nbsp;(<code>COUNT</code>, <code>SUM</code>, <code>AVG</code>, <code>MIN</code>, <code>MAX</code>)&nbsp; and <code>GROUP BY</code>.</li>
<li>Filter groups with <code>HAVING</code> and understand why it is different from <code>WHERE</code></li>
</ul>
</div>

<div class="alert alert-info">
<b>Prerequisites</b><br><br>
Before starting this workshop, you should:
<ul>
<li>Have basic familiarity with data analysis concepts</li>
<li>Have Python and Jupyter Notebook installed (optional, but helpful for following along)</li>
<li>Download the workshop materials (we'll provide these)</li>
<li>Not strictly necessary, but it would be very useful to have a notion of how to do operations on Pandas (or any other similar library/package/software) to better contextualize the material.</li>
</ul>

No prior SQL experience is required!
</div>

<div class="alert alert-warning">
<b>Setup Note</b><br><br>
If you want to follow along with the exercises: <br>
1. Make sure you have the required Python packages installed:<br>
   - pandas<br>
   - sqlite3<br>
   - sqlalchemy<br>
2. Download the example database file we'll be using
</div>

### Setup

In [14]:
# Importing packages that we will need
import sqlite3
from sqlalchemy import create_engine
import pandas as pd         
import os, pathlib, sys

# The first thing we need to do is connect to our SQLite database, which is the file that stores the information we will be using.
# If there is no pre-existing database with this name, it automatically creates one.
# Every SQL query will be executed through this `conn` object - which can be thought as a channel to the SQLite file.
conn = sqlite3.connect('../Data/customers.sqlite')
engine = create_engine('sqlite:///customers.sqlite')

# Load the customers table into a DataFrame for displ


# Troubleshooting: Please run the following cells to check whether sqlite is imported and the database file is in its appropriate location!

try:
    import sqlite3                        
    print("✅ The sqlite3 library is imported and available.")
except ImportError:
    print("❌ sqlite3 library not found.")

# 2️⃣ Check that the workshop DB file exists
from pathlib import Path
print("✅ The database file is available and in the appropriate folder" if Path("../Data/customers.sqlite").exists()
      else "❌ ../Data/customers.sqlite is missing—download the data bundle and place it there.")


✅ The sqlite3 library is imported and available.
✅ The database file is available and in the appropriate folder


### Icons Used in This Notebook
🔔 **Question**: A quick question to help you understand what's going on.<br>
🥊 **Challenge**: Interactive exercise. We'll work through these in the workshop!<br>
💡 **Tip**: How to do something a bit more efficiently or effectively.<br>
⚠️ **Warning:** Heads-up about tricky stuff or common mistakes.<br>
📝 **Poll:** A Zoom poll to help you learn!<br>
🎬 **Demo**: Showing off something more advanced – so you know what Python can be used for!<br>
🙋 Hands-Up: Quick pulse-check or mini-quiz. Respond by choosing the option that matches how you feel/what you think.

<a id='why-sql'></a>
## 1 · Why SQL? 

Estimated time: 15 minutes

A very common question when people learn about the existence of SQL is: Why bother learning it? Can't I just use pandas and pandas dataframes?

### Learning Objective
Understand when and why to use SQL instead of pandas, and how these tools complement each other in data analysis workflows.

📝 **Poll 1:** What is the usual size of databases you use? How large do you think databases can get?

### Bookcase vs Desk Analogy  
Think of your computer’s RAM as the desk where you spread papers you’re actively working on, and the hard‑drive as the bookcase that stores all your books.

Working at your desk is fast, but at the same time it doesn't hold nearly as much material as a bookshelf. So if you are working with very large datasets, you are at risk of overloading your desk with material. 

* **Pandas** is excellent for analysis of *smaller* data that fits comfortably on the desk.  
* **SQL** is the tool we use to selectively bring only the information we need from the bookcases to the desk.

🙋 **Hands-Up:** Did the analogy make sense? A. Yes B. Still fuzzy

 **Example:** A 20 GB transaction table can be grouped and aggregated with a single SQL statement, whereas pandas would first need 20 GB of memory just to read the file.

### Rough size sweet-spots on a single machine

- **Pandas (all in RAM)** – ideal up to **≈ 5 million rows** (around **500 MB** of CSV). Beyond that, Python overhead (≈ 5–10× the raw file size) quickly exhausts memory.  
- **SQLite (single-file engine)** – stays responsive with **tens or even hundreds of millions of rows** (roughly **10–500 GB**). Because it streams pages from disk, RAM is rarely the bottleneck. File-system ceiling: ≈ 280 TB.  
- **PostgreSQL (server engine)** – comfortably handles **100 GB to several terabyte tables** on industrial hardware; technical cap is **32 TB per table**, but overall database size is **unlimited**


### Complements, not Substitutes 
In practice we often:

1. Use SQL to slice/aggregate huge tables, producing a manageable result set  
2. Pull that into pandas for plotting or modelling

**💡Tip:** When working with large datasets, try to do as much filtering and aggregation as possible in SQL before pulling data into pandas. This can dramatically improve performance!

🔔 **Question:**  Have you ever dealt with a problem in which SQL was necessary? If not, can you think of one real life application that might require it?

<a id='relational'></a>
## 2 · Relational Databases 

Estimated time: 12 minutes

### Learning Objective
Grasp the fundamental concepts of relational databases, including tables, keys, and relationships between data.

## What is a Relational Database?

<div class="alert alert-primary">
<b>🔑 Key Concept: Relational Databases</b><br><br>
A relational database organizes data into tables (sometimes called relations), which consist of rows and columns:

- **Tables**: Collections of related data (e.g., Customers, Orders)
- **Columns**: Specific attributes (e.g., CustomerID, FirstName)
- **Rows**: Individual records in the table
- **Primary Keys**: Unique identifiers for each row
- **Foreign Keys**: References to primary keys in other tables that create relationships

![Database sample of Customers, Orders and OrderDetails tables, highlighting how relational data splits across tables.](../Images/relational-databases.svg)

Let's take a look at these example:
- Combined, these two tables form a database - a collection of data organized in tables.
- In this particular case, there are two tables, each one storing one type of information - either about Customers, or about Orders. But databases usually consist of many tables!
- Each table consists of rows - identifying individual records of information - and columns, which provide a piece of information for a given field corresponding to each row.
- Both of these tables have primary keys - columns that uniquely identify each row, i.e, each row has one, and no two rows share a primary key. Notice however the two tables do not have the same primary key!
- Further, the Order table has a foreign key "Customer ID" - a column linking to the primary key of the Customers table. Each order can be associated to a single customer through this relationship. But that doesn't mean each customer has just one order!

📝 **Poll 2:** Can you think of a table that would not have a Primary Key?


<a id='sqlite'></a>
## 3 · Getting Started with SQLite 

Estimated Time: 10 minutes

### Learning Objective
Understand how to set up SQLite, import data from CSV files, and establish a database connection for querying data.



## What is SQLite?

SQLite is a lightweight relational database. It is easy to install, use, and even though it doesn't have as many advanced capabilities as PostgreSQL or MySQL, it is a great way to start learning SQL. 

SQL queries are very similar (if not identical) across these programs, so there is a lot of transfer of knowledge if you decide to move on to another one in the future.

### When to Use Different Databases

| What You're Doing | SQLite | MySQL/PostgreSQL |
|-------------------|--------|------------------|
| Getting Started | Great for learning SQL | More complex to set up |
| Working Locally |  Just a file you can copy | Needs server installation |
| Small-Medium Projects |  Up to ~1TB of data | Any size |
| Data Science Projects |  Works great with Python | Needs more setup |
| Team Projects | Limited sharing options | Better for collaboration |

💡 **Real-World Examples**:
- **SQLite**: Your Jupyter notebooks, research projects, personal data analysis
- **MySQL/PostgreSQL**: Company databases, shared research data, production systems

The key takeaway: SQLite is perfect for learning SQL and doing your own data analysis. When you need to share data with a team or work on very large datasets, you can easily switch to MySQL or PostgreSQL - the SQL commands will be almost exactly the same!


📝 **Poll 3:** What's your primary reason for learning SQL today? 

### SQLite and Python

In this workshop, we'll work with SQLite directly in a Jupyter notebook using the `sqlite3` Python library, which comes built into Python.

We could create tables with `CREATE TABLE` statements, as well as add new data (INSERT), change existing data (UPDATE), and remove data (DELETE) directly on SQL, but in this workshop we will focus on querying data from a pre-existing database. I have added an example of how can this be done in an auxiliary file that I used to create the database for this workshop, if you would like to explore that later!


💡**Tip:** In this workshop we will be focusing on databases that contain a single table. In the next workshop, we will learn how to combine information from multiple tables. 

Let's upload our sql database as a dataframe in pandas:

customers_df = pd.read_sql_query('SELECT * FROM customers', conn)



🥊 **Challenge**: Can you use pandas to take a look at what this dataset looks like?

In [8]:
### SOLUTION:
customers_df.head()    

,name,city,country,items_purchased,price_per_item,last_purchase,account_balance
0,John Smith,New York,US,NaN,56.28,None,945.55
1,Maria Garcia,London,GB,15.0,NaN,2024-08-18,905.34
2,Li Wei,Tokyo,JP,11.0,14.18,2024-02-10,638.11
3,Emma Brown,Paris,FR,8.0,64.68,2024-01-28,929.69
4,Ahmed Hassan,Sydney,AU,7.0,25.35,2024-05-14,179.64


<a id='types'></a>
## 4 · SQLite Data Types & `NULL` 

Estimated time: 7 minutes

### Learning Objective
Understand SQLite's data types and how NULL values are handled in SQL databases.

SQLite has a flexible approach to data types. While columns are designed with a recommended data type (like TEXT or INTEGER), SQLite can actually store any type of data in any column.

* **INTEGER** – whole numbers 
* **REAL** – floating‑point  
* **TEXT** – strings    
* **NULL** – missing / undefined  

Unlike PostgreSQL/MySQL, SQLite does NOT have a date format, but it does have dedicated functions that interpret strings as dates.

<a id='select'></a>
## 5 · SELECT & Derived Columns 

Estimated time: 18 minutes

### Learning Objective
Master the fundamental SQL SELECT statement, including column selection, aliasing, and creating derived columns using basic SQL functions.

Real-World Application: Extract the relevant customer information for a marketing campaign

### The SELECT statement is the foundation of SQL queries. We'll learn it step by step:

1. Basic SELECT - Retrieving data from tables
2. Column Aliases - Giving friendly names to columns
3. Derived Columns - Creating new columns from existing ones
4. SQL Functions - Transforming data in useful ways

💡 Tip: We'll build from simple queries to more complex ones, making sure each concept is clear before moving on.

At its most basic version, a query looks like this:

```sql
SELECT column1, column2, …
FROM   table;
```

- One of the most confusing aspects of SQL, especially in the beginning, is understading the order in which SQL reads the instructions of a query - which unfortunately is not the order in which the commands are written.
- Here it is better to think that the first command is "FROM" - which tells us where to draw the information from
- After that, we can select only the columns we will need by listing them after the SELECT statement.
- Notice that:
    - We don't need to put names of columns and tables between quotes
    - We do need to separate different columns by commas - but not one at the end!

🙋 **Hands-Up:** Which clause executes *first* in SQL’s logical order? A. `SELECT` B. `FROM` C. `WHERE`

💡 **Tip**: When selecting multiple columns, use line breaks after commas for better readability!

In [3]:
# Basic SELECT query
query = """
SELECT name, city, country, items_purchased
FROM customers
"""
result = pd.read_sql_query(query, conn)
print("Basic SELECT:\n")
print(result)

Basic SELECT:

                name         city country  items_purchased
0         John Smith     New York      US              NaN
1       Maria Garcia       London      GB             15.0
2             Li Wei        Tokyo      JP             11.0
3         Emma Brown        Paris      FR              8.0
4       Ahmed Hassan       Sydney      AU              7.0
5      Sarah Johnson       Berlin      DE             19.0
6   Carlos Rodriguez       Mumbai      IN             11.0
7      Anna Kowalski    São Paulo      BR             11.0
8       James Wilson      Toronto      CA              4.0
9        Yuki Tanaka     Shanghai      CN              8.0
10       Elena Popov       Madrid      ES              3.0
11     Michel Dubois       Moscow      RU              2.0
12      Sofia Santos        Dubai      AE             12.0
13     Lars Andersen         None    None              6.0
14       Aisha Patel  Mexico City      MX              2.0
15    Diego Martinez    Amsterdam      NL

⚠️ **Warning:** Common Mistake - Missing commas between columns
```sql
-- ❌ WRONG: Missing commas
SELECT name city country
FROM customers

-- ✅ CORRECT: Columns separated by commas
SELECT name, city, country
FROM customers

However, queries are extremely flexible and allow us to combine/summarize information in many ways.

Let's take a look at some examples of more interesting queries

### Step‑by‑Step Examples

1. **Select everything:**


   If you want to keep all columns from a table, you can just use * instead of naming the columns.


   ```sql
   SELECT * 
   FROM customers;
   ```

In [4]:
# Select all columns example
select_all_query = """
SELECT * 
FROM customers
"""
result = pd.read_sql_query(select_all_query, conn)
print("SELECT * - All columns:\n")
print(result)

SELECT * - All columns:

                name         city country  items_purchased  price_per_item  \
0         John Smith     New York      US              NaN           56.28   
1       Maria Garcia       London      GB             15.0             NaN   
2             Li Wei        Tokyo      JP             11.0           14.18   
3         Emma Brown        Paris      FR              8.0           64.68   
4       Ahmed Hassan       Sydney      AU              7.0           25.35   
5      Sarah Johnson       Berlin      DE             19.0           15.85   
6   Carlos Rodriguez       Mumbai      IN             11.0           95.40   
7      Anna Kowalski    São Paulo      BR             11.0           96.91   
8       James Wilson      Toronto      CA              4.0           82.76   
9        Yuki Tanaka     Shanghai      CN              8.0           37.42   
10       Elena Popov       Madrid      ES              3.0           18.79   
11     Michel Dubois       Moscow      

🔔 **Question:** When might using SELECT * be problematic in a real database?

### 5.2 Column Aliases

    Sometimes tables have very complicated/uninformative/ambiguous names. We can rename the header of a column using an alias, which is a new name to columns in our result. This is useful when:
    - Making technical names more readable
    - Clarifying the meaning of computed columns
    - Creating reports for non-technical users

Syntax using AS (recommended for clarity):

   ```sql
   SELECT column AS new_name,
   FROM   table;
   ```

Alternative syntax (implicit):

   ```sql
   SELECT column new_name,
   FROM   table;
   ```

⚠️ **Warning:** If your new name contains spaces or is a special name, you must wrap it in quotes:

   ```sql
   SELECT column AS "Full Name",
   FROM   table;
   ```

In [5]:
# Column aliases example
alias_query = """
SELECT 
    name AS customer_name,
    city AS location,
    items_purchased AS quantity,
    price_per_item AS unit_price
FROM customers
WHERE items_purchased IS NOT NULL
"""
result = pd.read_sql_query(alias_query, conn)
print("SELECT with Aliases: \n")
print(result)

SELECT with Aliases: 

       customer_name     location  quantity  unit_price
0       Maria Garcia       London      15.0         NaN
1             Li Wei        Tokyo      11.0       14.18
2         Emma Brown        Paris       8.0       64.68
3       Ahmed Hassan       Sydney       7.0       25.35
4      Sarah Johnson       Berlin      19.0       15.85
5   Carlos Rodriguez       Mumbai      11.0       95.40
6      Anna Kowalski    São Paulo      11.0       96.91
7       James Wilson      Toronto       4.0       82.76
8        Yuki Tanaka     Shanghai       8.0       37.42
9        Elena Popov       Madrid       3.0       18.79
10     Michel Dubois       Moscow       2.0       71.58
11      Sofia Santos        Dubai      12.0       49.61
12     Lars Andersen         None       6.0       20.98
13       Aisha Patel  Mexico City       2.0       54.57
14    Diego Martinez    Amsterdam       1.0       13.09
15         Lucy Chen        Cairo      12.0       91.84
16       Ivan Petrov    S

### 5.3 Derived Columns

Very commonly we might want to create what we call a "Derived Column" - which is a column that modifies/combines information from columns of the original table. 

The way that we do this is usually by using functions. The syntax is function(column_name). We treat them as if they were regular columns.

Common Uses:
- Perform calculations
- Combine text
- Transform data

Basic syntax:
```sql
SELECT 
    original_column,
    expression AS new_column
FROM table;
```

Common uses:
1. Arithmetic: `price * quantity AS total`
2. Text concatenation: `first_name || ' ' || last_name AS full_name`
3. Simple calculations: `price * 1.2 AS price_with_tax`

![Workflow diagram showing SQL string concatenation that creates a “summary” column from name, price and quantity fields.](../Images/derivedcolumn.svg)
---

In [6]:
# Derived columns with string functions
select_derived_query = """
SELECT 
    LOWER(name) AS lower_name,
    UPPER(city) AS upper_city,
    items_purchased * 2 AS double_items
FROM customers
"""
print("SELECT Example with Derived Columns: \n")
print(pd.read_sql_query(select_derived_query, conn))

SELECT Example with Derived Columns: 

          lower_name   upper_city  double_items
0         john smith     NEW YORK           NaN
1       maria garcia       LONDON          30.0
2             li wei        TOKYO          22.0
3         emma brown        PARIS          16.0
4       ahmed hassan       SYDNEY          14.0
5      sarah johnson       BERLIN          38.0
6   carlos rodriguez       MUMBAI          22.0
7      anna kowalski    SãO PAULO          22.0
8       james wilson      TORONTO           8.0
9        yuki tanaka     SHANGHAI          16.0
10       elena popov       MADRID           6.0
11     michel dubois       MOSCOW           4.0
12      sofia santos        DUBAI          24.0
13     lars andersen         None          12.0
14       aisha patel  MEXICO CITY           4.0
15    diego martinez    AMSTERDAM           2.0
16         lucy chen        CAIRO          24.0
17       ivan petrov    STOCKHOLM          24.0
18     mary williams         None          34.0
1

⚠️ Warning: Common Mistake - Using double quotes for string values

```sql
--❌ WRONG: Double quotes for string values
WHERE country = "USA"

-- ✅ CORRECT: Single quotes for strings
WHERE country = 'USA'
```

In [7]:
# Combining columns and calculations
select_combined_query = """
SELECT 
    name,
    city || ', ' || country AS full_location,
    items_purchased * price_per_item AS total_spent
FROM customers
"""
print("SELECT Example with Combined Columns: \n")
print(pd.read_sql_query(select_combined_query, conn))

SELECT Example with Combined Columns: 

                name    full_location  total_spent
0         John Smith     New York, US          NaN
1       Maria Garcia       London, GB          NaN
2             Li Wei        Tokyo, JP       155.98
3         Emma Brown        Paris, FR       517.44
4       Ahmed Hassan       Sydney, AU       177.45
5      Sarah Johnson       Berlin, DE       301.15
6   Carlos Rodriguez       Mumbai, IN      1049.40
7      Anna Kowalski    São Paulo, BR      1066.01
8       James Wilson      Toronto, CA       331.04
9        Yuki Tanaka     Shanghai, CN       299.36
10       Elena Popov       Madrid, ES        56.37
11     Michel Dubois       Moscow, RU       143.16
12      Sofia Santos        Dubai, AE       595.32
13     Lars Andersen             None       125.88
14       Aisha Patel  Mexico City, MX       109.14
15    Diego Martinez    Amsterdam, NL        13.09
16         Lucy Chen        Cairo, EG      1102.08
17       Ivan Petrov    Stockholm, SE     

🔔 **Question:** Why are we getting NaN's in the last column? How can we avoid this issue?

<div class="alert alert-info">
<b>💡 Best Practices</b><br><br>
1. Always explicitly list the columns you need <br>
2. Use meaningful aliases for clarity<br>
3. Format queries with proper indentation<br>
4. Add comments for complex queries<br>
5. Test queries with a subset of data first
</div>

### Commonly used Functions

We won't be able to cover all of them, but for future reference, here is a quick list of commonly used functions:

#### Text functions

| Function | Description | Example |
|----------|-------------|---------|
| `SUBSTR(text, start, len)` | Substring (1‑based index) | `SUBSTR('Market',1,3)` → `Mar` |
| `INSTR(text, pattern)` | Position (0 if not found) | `INSTR('abcdef','cd')` → `3` |
| `LOWER(text)` / `UPPER(text)` | Case conversion | `LOWER('SQL')` → `sql` |
| `REPLACE(text, old, new)` | Global substitution | `REPLACE('foo','o','0')` |
| `TRIM(text)` | Strip leading/trailing spaces | |
| `str1 \|\| ' merging character ' \|\| str2 \|\|` | Concatenates strings

#### Date / time Functions

| Function | What it does | Example |
|----------|--------------|---------|
| `DATE('now')` | Current date (UTC) | `DATE('now')` → `'2025‑05‑07'` |
| `DATETIME('now','localtime')` | Current local datetime | `DATETIME('now','localtime')` → `'2025‑05‑07 13:25:00'` |
| `STRFTIME(fmt, ts)` | Format timestamp → text | `STRFTIME('%Y‑%m', OrderDate)` → `'1997‑07'` |
| `JULIANDAY(ts)` | Days since noon 4714 BC | `JULIANDAY('2025‑05‑07')` → `2460457.5` |


#### Integer and Float Functions

| Expression | Result |
|------------|--------|
| `column1 + column2` | Add two numbers |
| `ROUND(total * 0.15, 2)` | Round to 2 decimal places |
| `COALESCE(price, 0)` | Replace NULL with 0 |

💡**Tip:** Need a float? Multiply an `INTEGER` by **`1.0`**.

Now that we know the basics of querying, we can start dealing with more complex query commands - such as filtering, grouping, aggregate functions, sorting and paginating.

<a id='where'></a>
## 6 · Filtering Rows with `WHERE` 

Esimated time: 10 minutes

### Learning Objective
Learn how to filter data using WHERE clauses, including comparison operators, logical operators (AND, OR), and pattern matching with LIKE.

🔑 **Key Concept:** Filtering

Filtering is the process of selecting a subset of rows that match a certain condition. Think of it as answering the question "Which rows do I want to keep?"

Filtering happens before selecting. This is at the crux of our desk-bookshelf analogy - we just want to query the data that we will be needing, so that it is manageable when it gets to our desk.


**📚 Real-World Applications**
- Finding transactions above a certain amount for audit <br>
- Filtering customer data by region for targeted marketing<br>
- Identifying high-value products for inventory management<br>
- Finding recent orders for shipping prioritization<br>



The basic filtering method is WHERE, and the syntax is as follows:

```sql
SELECT columns
FROM table
WHERE condition

```

As we will see in a bit, the conditions we can impose are very flexible. But, as a first example, let's filter our previous database to those consumers who purchased more than one item.

In [8]:
# Customers who purchased more than one item
where_query = """
SELECT name, city, items_purchased, price_per_item
FROM customers
WHERE items_purchased > 1
"""
result = pd.read_sql_query(where_query, conn)
print("Customers who purchased more than one item:")
print(result)

Customers who purchased more than one item:
                name         city  items_purchased  price_per_item
0       Maria Garcia       London             15.0             NaN
1             Li Wei        Tokyo             11.0           14.18
2         Emma Brown        Paris              8.0           64.68
3       Ahmed Hassan       Sydney              7.0           25.35
4      Sarah Johnson       Berlin             19.0           15.85
5   Carlos Rodriguez       Mumbai             11.0           95.40
6      Anna Kowalski    São Paulo             11.0           96.91
7       James Wilson      Toronto              4.0           82.76
8        Yuki Tanaka     Shanghai              8.0           37.42
9        Elena Popov       Madrid              3.0           18.79
10     Michel Dubois       Moscow              2.0           71.58
11      Sofia Santos        Dubai             12.0           49.61
12     Lars Andersen         None              6.0           20.98
13       Aisha Pat

This example actually shows something very interesting about the ```WHERE``` statement - we don't need to keep the column we are using to filter the rows.

In other words, let's say that we don't care too much about how much an user actually purchase, but we only want those who spend above a threshold. We can use ```WHERE``` combined with ```SELECT``` to use the information for filtering, but not keep it with us after it has been used, saving a lot of memory.

🥊 **Challenge:** List the `name`, `country`, and `age` of all customers **older than 50** who live in **Brazil**.

In [ ]:
# This will raise a sqlite3.OperationalError — fix it!
bad_query = """
SELECT name country age
FROM customers
WHERE age > 50
  country = 'Brazil';
"""
pd.read_sql_query(bad_query, conn)

### 6.1 Filtering Methods

Here are some commonly used filtering methods:

1. **Equality / inequality**  
   ```sql
   … WHERE country = 'Germany';
   ```
⚠️ **Warning:** Be careful with string comparisons - they might be case-sensitive depending on the SQL engine and version you are using.

2. **Set membership**  
   ```sql
   … WHERE country IN ('USA','UK','Germany');
   ```

3. **Comparison**  
   ```sql
   … WHERE freight > 100;
   ```

📝 **Poll 4:** Which comparison operator would you use to find values in a specific range?

4. **NULL checks**  
   ```sql
   … WHERE fax IS NULL;
   ```

⚠️ **Warning:** When you compare anything to NULL, the result isn't TRUE or FALSE, it is a special third type called UNKNOWN. 

This feature can be quite confusing, especially in the beginning. A common mistake is to try to find rows with missing values by using 

'WHERE column = NULL'

This leads to the comparison  `NULL` **≠** `NULL` , which yields *UNKNOWN*, rather than TRUE.

In SQL, we should instead use dedicated functions:

```sql
… WHERE fax IS NULL
… WHERE fax IS NOT NULL
```

<div class="alert alert-warning">
<b>⚠️ Other common WHERE mistakes</b><br><br>

1. String matching
   - ❌ Forgetting quotes: `WHERE name = John`
   - ✅ Using quotes: `WHERE name = 'John'`

2. Date comparisons
   - ❌ `WHERE date >= '2024-01-01'` AND `date <= '2024-12-31'` 
   - ✅ `WHERE date >= '2024-01-01'` AND `date < '2025-01-01'`
</div>


In [9]:
# Demonstrating NULL handling with our customers data
# First, the incorrect way
incorrect_null_query = """
SELECT name, city
FROM customers
WHERE city = NULL
"""
print("Incorrect NULL check (city = NULL):")
print(pd.read_sql_query(incorrect_null_query, conn))

# Now the correct way
correct_null_query = """
SELECT name, city, country
FROM customers
WHERE city IS NULL
"""
print("Correct NULL check (city IS NULL):")
print(pd.read_sql_query(correct_null_query, conn))

Incorrect NULL check (city = NULL):
Empty DataFrame
Columns: [name, city]
Index: []
Correct NULL check (city IS NULL):
            name  city country
0  Lars Andersen  None    None
1  Mary Williams  None    None


### 6.2 Multiple Conditions
We can combine multiple conditions using logical operators:

- AND: Both conditions must be true
- OR: At least one condition must be true
- NOT: Reverses a condition
Use parentheses to make the order of operations clear:

```sql
WHERE (country = 'US' OR country = 'GB')
  AND account_balance > 200
```


💡 **Tip:** You can combine multiple logical expressions using AND/OR/NOT. When doing so, use parentheses to clarify when one logical expressions begins and the other ends.
⚠️ **Warning:** Without parentheses, AND takes precedence over OR.

<a id='groupby'></a>
## 7 · Aggregate Functions & `GROUP BY` 

Estimated Time: 25 minutes

### Learning Objective
Master data aggregation and grouping using SQL's aggregate functions (COUNT, SUM, AVG, etc.) and GROUP BY clause to analyze and summarize data effectively.

### Real World Application  
Given a list of individual transactions, calculate total sales per region.

### 7.1 The Basic Idea

Another way of pre-processing data so that the end result is more manageable is to summarize it according to a given statistic.

One common example is the use of aggreggate functions, combined with GROUP BY, to collapse many rows into one, while keeping the information contained in them, only now summarized in a single row.

First let's understand what the ```GROUP BY``` statement does: it creates subsets of the entire table that are similar in a given way. The most common way of doing so is to pass a column - and then SQL will automatically group the rows according to the values in that column

Second, ```GROUP BY``` statements are used in conjunction with aggreggate functions. By grouping rows according to a given column, we can guarantee that the values of these rows match **for that particular column**. But what about the others? They might be different, in which case there is no obvious way of combining them. Aggregate functions do exactly this - they tell SQL what to do with mismatching information inside the group - for example by counting the number of occurrences, summing or taking averages:

```sql
SELECT country,
       COUNT(*)        AS n_orders,
       ROUND(AVG(freight),2) AS avg_freight
FROM   orders
GROUP  BY country
```


![Three-step graphic: raw sales rows, grouped by Department, then totals—illustrates how GROUP BY collapses data.](../Images/groupby.svg) 

Let's break it down what is going on with the GROUP BY command.

- First, SQL will look into the column indicated on GROUP BY - in this case "department"
- It will then create buckets given the entries in this column. In other words, for each possible value in this column, it will group the rows based on these values.
- For each of these groups, it will run an aggregate function - in this case COUNT(*), which counts how many entries each group has, and SUM(salary), which will sum the column salary across all rows in a given group
- It will finally return the values of each group, and of the aggregate functions. Notice that each group only has one row in the resulting column - we have "collapsed" the table!


🥊 **Challenge:** For each country, list the number of customers (COUNT(*)) and the total items purchased (SUM(items_purchased)).
Show only countries with at least 5 customers.

In [ ]:
# Debug this intentional error

bad_query = """
SELECT country,
       COUNT(*),                       
       SUM(items_purchased)
FROM customers
WHERE COUNT(*) >= 5                    
GROUP BY country;
"""
pd.read_sql_query(bad_query, conn)

### 7.2 More Advanced Ideas
Another very important thing - which is a bit tough to get used to in the beginning, is that we can only include in the SELECT statement columns that are either used to group by observations, or ones that are used as inputs of aggregate functions. This is exactly because of what we discussed previously - if there is a mismatch between rows, SQL doesn't know how to handle these values when it collapses all the rows in the group into a single one.

A bit more advanced, but we can also pass more than one column to the GROUP BY statement - which would then create groups in which rows have the same values for all columns passed.

💡 **Tip**: By using GROUP BY, we obtain a collapsed version of the table - we only retain information on the aggregate values. While this is very useful for summarizing information, sometimes we want to keep the detailed data and the summary statistics for more in-depth analysis. This is exactly the problem that Window Functions solve - and something we will be dealing with in the intermediate workshop!

In [10]:
# GROUP BY country with aggregations
group_by_query = """
SELECT 
    country,
    COUNT(*) AS customer_count,
    ROUND(AVG(items_purchased), 2) AS avg_items,
    ROUND(AVG(account_balance), 2) AS avg_balance,
    ROUND(MAX(account_balance), 2) AS max_balance
FROM customers
WHERE country IS NOT NULL
GROUP BY country
"""
result = pd.read_sql_query(group_by_query, conn)
print("Customer statistics by country:\n")
print(result)

Customer statistics by country:

   country  customer_count  avg_items  avg_balance  max_balance
0       AE               1       12.0       352.84       352.84
1       AU               1        7.0       179.64       179.64
2       BR               1       11.0       392.80       392.80
3       CA               1        4.0       449.81       449.81
4       CN               1        8.0       344.21       344.21
5       DE               1       19.0          NaN          NaN
6       EG               1       12.0       167.10       167.10
7       ES               1        3.0       845.86       845.86
8       FR               1        8.0       929.69       929.69
9       GB               1       15.0       905.34       905.34
10      HK               1       15.0       833.92       833.92
11      IN               1       11.0       140.70       140.70
12      IT               1       16.0       104.97       104.97
13      JP               1       11.0       638.11       638.11
14     

⚠️ Warning: Common Mistake - Using aggregate functions without GROUP BY

```sql
-- ❌ WRONG: Mixing aggregate and non-aggregate columns
SELECT country, COUNT(*), AVG(account_balance)
FROM customers

-- ✅ CORRECT: Add GROUP BY for non-aggregate columns
SELECT country, COUNT(*), AVG(account_balance)
FROM customers
GROUP BY country
```

### 7.3: Summarizing

Key points:

- Every non-aggregated column in SELECT must be in GROUP BY
- GROUP BY comes after WHERE but before ORDER BY
- Can group by multiple columns
- Can use expressions in GROUP BY 

For future reference, here is a list of the most commonly used aggregate functions:

| Aggregate Function                                   | What it returns (typical usage)                    |
| ---------------------------------------------------- | -------------------------------------------------- |
| `COUNT(*)`                                           | Total number of rows in the group/query            |
| `SUM(col)`                                           | Arithmetic sum of a numeric column                 |
| `AVG(col)`                                           | Mean (average) of numeric values                   |
| `MAX(col)`                                           | Largest value (numeric *or* lexicographic)         |
| `MIN(col)`                                           | Smallest value (numeric *or* lexicographic)        |
| `STRING_AGG(col, ', ')` / `GROUP_CONCAT` / `LISTAGG` | Concatenates strings in the group with a delimiter |
| `COUNT(DISTINCT col)`                                | Count of unique, non-NULL values                   |

💡 **Tip**: Mathematical functions ignore `NULL` in aggregates (`AVG`, `SUM`), which is usually what you want. `COUNT(column)` counts only non‑null values, whereas `COUNT(*)` counts *all* rows

### 7.4 · Filtering Groups with `HAVING` 

Remember that when we discussed filtering, we used the WHERE command, which was run before the ```GROUP BY```.

Sometimes, we want to filter rows given an aggregate statement. For example, we might want to choose only the customers whose average expenditure is larger than a certain amount.

`HAVING` is evaluated **after** grouping – it filters *groups*, whereas `WHERE` filters *rows*.

Two Observations:
- We cannot use aggregate functions on ```WHERE``` statements
- We must use ```HAVING``` after the ```GROUP BY``` statement

```sql
SELECT country,
       COUNT(*) AS n_orders
FROM   orders
GROUP  BY country
HAVING n_orders > 20          -- aggregate in condition
ORDER  BY n_orders DESC;
```

In [11]:
# Countries with high average purchases using HAVING
having_query = """
SELECT 
    country,
    COUNT(*) AS customer_count,
    ROUND(AVG(items_purchased), 2) AS avg_items_purchased,
    ROUND(AVG(account_balance), 2) AS avg_balance
FROM customers
WHERE country IS NOT NULL 
  AND items_purchased IS NOT NULL
GROUP BY country
HAVING AVG(items_purchased) > 5 
   AND COUNT(*) >= 2
"""
result = pd.read_sql_query(having_query, conn)
print("Countries with average purchases > 5 and at least 2 customers:")
print(result)

Countries with average purchases > 5 and at least 2 customers:
Empty DataFrame
Columns: [country, customer_count, avg_items_purchased, avg_balance]
Index: []


![Split-path illustration contrasting filtering rows with WHERE before grouping versus HAVING after aggregation.](../Images/wherehaving.svg)

🙋 **Hands-Up:** Can you explain—in one sentence—what `HAVING` does that `WHERE` can’t? A. Yes B. Not yet

<a id='orderby'></a>
## 8 · Sorting & Paginating Results 

Estimate Time: 10 minutes

### Real World Application: 

Find the top 10 selling items on a given year.

After we have processed our data, we might want to start preparing it for visualization. In SQL, this is done mostly through sorting - ordering the data according to one or more columns - or paginating - retrieving only a fixed number of observations

`ORDER BY` is evaluated *after* `SELECT`.  

* Default sort is **ASC** (ascending).  
* Use **DESC** for descending order.  


In [12]:
# Simple ORDER BY example with one column
simple_order_query = """
SELECT 
   name,
   city,
   items_purchased
FROM customers
WHERE items_purchased IS NOT NULL
ORDER BY items_purchased DESC
"""

print("Simple ORDER BY Example - Top 10 Customers by Items Purchased:\n")
result = pd.read_sql_query(simple_order_query, conn)
print(result)

Simple ORDER BY Example - Top 10 Customers by Items Purchased:

                name         city  items_purchased
0      Sarah Johnson       Berlin             19.0
1           Jun Park        Seoul             19.0
2      Mary Williams         None             17.0
3       Hans Schmidt         Rome             16.0
4       Maria Garcia       London             15.0
5     Isabella Silva    Hong Kong             15.0
6       Sofia Santos        Dubai             12.0
7          Lucy Chen        Cairo             12.0
8        Ivan Petrov    Stockholm             12.0
9       Anna Ivanova      Bangkok             12.0
10            Li Wei        Tokyo             11.0
11  Carlos Rodriguez       Mumbai             11.0
12     Anna Kowalski    São Paulo             11.0
13         Raj Kumar  Los Angeles             10.0
14        Emma Brown        Paris              8.0
15       Yuki Tanaka     Shanghai              8.0
16      Ahmed Hassan       Sydney              7.0
17     Lars Anders

You can order by *multiple* columns – the second acts as a tie‑breaker.

```sql
SELECT
       name,
       country,
       total_spent,
       ROW_NUMBER() OVER (
           PARTITION BY country
           ORDER BY total_spent DESC,   -- primary
                    name               -- secondary tie-breaker
       ) AS spend_rank
FROM   customer_spending;
```


In [13]:
# Example showing ORDER BY with multiple columns
order_by_example = """
SELECT 
   name,
   country,
   account_balance
FROM customers
WHERE account_balance IS NOT NULL
ORDER BY country ASC, account_balance DESC
LIMIT 10;
"""

print("ORDER BY Example - Sorting by Country then Balance:")
result = pd.read_sql_query(order_by_example, conn)
print(result)

ORDER BY Example - Sorting by Country then Balance:
            name country  account_balance
0  Mary Williams    None           795.02
1  Lars Andersen    None           588.43
2   Sofia Santos      AE           352.84
3   Ahmed Hassan      AU           179.64
4  Anna Kowalski      BR           392.80
5   James Wilson      CA           449.81
6    Yuki Tanaka      CN           344.21
7      Lucy Chen      EG           167.10
8    Elena Popov      ES           845.86
9     Emma Brown      FR           929.69



### Pagination Pattern

LIMIT is used to restrict how many observations we want to retrieve <br>
OFFSET will skill a certain number of rows before displaying the number of results delimited by LIMIT


```sql
SELECT company_name, country, city
FROM   customers
ORDER  BY country
LIMIT  n
OFFSET m;
```

`LIMIT` must appear *before* `OFFSET` in SQLite.

⚠️ **Warning:** It is very hard to predict what the ordering will be after applying filtering or other methods. So remember to always ORDER BY before using LIMIT/OFFSET!

💡 **Tip**: Interestingly, LIMIT doesn't restrict the data being retrived by SQL, only the data being showed. The difference is crucial to understand when thinking about factors such as speed, memory constraints and computing budgeting!

In [14]:
# Sorting and pagination example
pagination_query = """
SELECT 
    name,
    country,
    account_balance
FROM customers
WHERE account_balance IS NOT NULL
ORDER BY account_balance DESC
LIMIT 5
OFFSET 5;
"""
result = pd.read_sql_query(pagination_query, conn)
print("Customers ranked 6-10 by account balance:\n")
print(result)

Customers ranked 6-10 by account balance:

             name country  account_balance
0  Isabella Silva      HK           833.92
1  Diego Martinez      NL           821.98
2   Mary Williams    None           795.02
3    Anna Ivanova      TH           794.14
4        Jun Park      KR           756.11


🙋 **Hands-Up:** Which clause actually *sorts* the result set? A. `SELECT` B. `ORDER BY` C. `LIMIT`

⚠️ Warning: Common Mistake - Wrong clause order

-- ❌ WRONG: ORDER BY must come before LIMIT
```sql
SELECT * FROM customers
LIMIT 10
ORDER BY account_balance DESC 
```

-- ✅ CORRECT: Proper SQL clause order
```sql
SELECT * FROM customers
ORDER BY account_balance DESC
LIMIT 10 ```

## Putting it all together

We learned quite a few different commands for queries. Let's see one example that includes all of them:

```sql
SELECT
    Country,
    COUNT(OrderID)                AS total_orders,
    ROUND(AVG(Freight), 2)        AS avg_freight
FROM    customers 
WHERE   Country IN ('USA','UK','Germany')
GROUP BY Country
HAVING   COUNT(OrderID) >= 10
ORDER BY total_orders DESC
LIMIT 5;
```

The diagram reiterates the **logical query order**, not the command order, helping you remember the order in which the operations are actually made.

![Horizontal flowchart listing SQL clause execution order: FROM → WHERE → GROUP BY → HAVING → SELECT → ORDER BY → LIMIT](../Images/sql-execution-order.svg)
---




### A visualization of the order of query operations

Let's go through an example of how the order of query operations look like in practice. 

**FROM `customers`** – full table (7 rows).

| CustID | Country | Orders |
|-------|---------|--------|
| C1 | USA | 5 |
| C2 | USA | 7 |
| C3 | UK  | 3 |
| C4 | UK  | 7 |
| C5 | FRA | 15 |
| C6 | GER | 2 |
| C7 | CAN | 6 |

---

**WHERE `Country IN ('USA','UK','FRA','GER')`** – drop the Canadian row.

| CustID | Country | Orders |
|-------|---------|--------|
| C1 | USA | 5 |
| C2 | USA | 7 |
| C3 | UK  | 3 |
| C4 | UK  | 7 |
| C5 | FRA | 15 |
| C6 | GER | 2 |

---

**GROUP BY `Country`** – aggregate rows, summing `Orders` - Drops CustID!

| Country | TotalOrders |
|---------|-------------|
| USA | 12 |
| UK  | 10 |
| FRA | 15 |
| GER | 2  |

---

**HAVING `TotalOrders > 5`** – keep only groups with large order volume; Germany drops out.

| Country | TotalOrders |
|---------|-------------|
| FRA | 15 |
| USA | 12 |
| UK  | 10 |

---

**SELECT `Country, TotalOrders`** – project just the columns we care about (already those two).

| Country | TotalOrders |
|---------|-------------|
| FRA | 15 |
| USA | 12 |
| UK  | 10 |

---

**ORDER BY `TotalOrders DESC`** – rank countries by order volume.

| Country | TotalOrders |
|---------|-------------|
| FRA | 15 |
| USA | 12 |
| UK  | 10 |

---

**LIMIT 2** – return only the top two performers.

| Country | TotalOrders |
|---------|-------------|
| FRA | 15 |
| USA | 12 |





<a id='keypoints'></a>
<div class="alert alert-success">  
    
## 11 · Key Points & Next Steps 

Estimate Time: 2 minutes

* Use SQL to select and pre-process only the data you really need, then use this smaller dataset for analysis with pandas
* SQLite is a zero‑install, single‑file engine that still speaks standard SQL.  
* Remember the logical query order to avoid confusion (`WHERE` vs `HAVING`).  

### What’s Next?  
In the **Intermediate SQL** workshop we will tackle:

* Creating & altering tables  
* `JOIN`s (```INNER```, ```LEFT```, ```RIGHT```, ```FULL```) and set operations
* `JOIN` as selection  
* Subqueries & Common Table Expressions (`WITH`)  
* Window functions (`ROW_NUMBER`, `LAG`, `LEAD`)  
* ```UNION```
* Pivoting

</div>

## 🎬 Demo — Customer Spending vs. Income Ranking  

💡 **Goal**  
Show how window functions, CTEs, and JOINs can be combined to answer a practical business question:

> *“How much does each customer spend relative to the income they report?”*

### What the query does
1. **`customer_spending` CTE**  
   *Calculates* each customer’s total spending (`items_purchased × price_per_item`).

2. **`income_ranking` CTE**  
   Aggregates income **and** ranks customers by `SUM(amount)` using  
   `RANK() OVER (ORDER BY total_income DESC)`.

3. **Main SELECT**  
   *Joins* the two CTEs on `name` and computes  
   **`spending_pct_of_income`** = spending ÷ income × 100.

Run the cell below and inspect the result.


In [15]:
# DEMO: Combining Window Functions with JOIN
income_df = pd.read_sql_query('SELECT * FROM income', conn)

demo_query = """
WITH customer_spending AS (
    SELECT 
        name,
        items_purchased * price_per_item AS total_spent
    FROM customers
    WHERE items_purchased IS NOT NULL 
      AND price_per_item IS NOT NULL
),
income_ranking AS (
    SELECT 
        name,
        SUM(amount) AS total_income,
        RANK() OVER (ORDER BY SUM(amount) DESC) AS income_rank
    FROM income
    WHERE amount IS NOT NULL
    GROUP BY name
)
SELECT 
    cs.name,
    cs.total_spent,
    ir.total_income,
    ir.income_rank,
    CASE 
        WHEN ir.total_income > 0 
        THEN ROUND(cs.total_spent * 100.0 / ir.total_income, 2)
        ELSE NULL 
    END AS spending_pct_of_income
FROM customer_spending cs
JOIN income_ranking ir ON cs.name = ir.name
ORDER BY ir.income_rank;
"""

print("🎬 DEMO: Customer Spending Analysis with Income Ranking\n")
display(pd.read_sql_query(demo_query, conn))


🎬 DEMO: Customer Spending Analysis with Income Ranking



,name,total_spent,total_income,income_rank,spending_pct_of_income
0,Ahmed Hassan,177.45,5800.0,1,3.06
1,Lars Andersen,125.88,4500.0,2,2.80
2,Yuki Tanaka,299.36,3200.0,8,9.36
3,Diego Martinez,13.09,2700.0,10,0.48
4,Elena Popov,56.37,1800.0,12,3.13
5,Mary Williams,1183.71,1200.0,13,98.64
6,Sarah Johnson,301.15,400.0,14,75.29
7,Li Wei,155.98,200.0,15,77.99


🙋 **Hands-Up:** How was that exercise? A. Easy B. Okay C. Tough


---
# Workshop 2 – SQL for Data Analysis (Part 2)


# SQL for Data Analysis  
## Advanced Workshop
*D‑Lab, UC Berkeley*

<div class="alert alert-success"> 
<b>Prerequisites for SQL Advanced Workshop</b><br><br>
Completion of "SQL for Data Analysis: Introductory Workshop" or equivalent experience:
<ul>
<li>Understanding of basic SQL syntax including SELECT, FROM, WHERE, GROUP BY</li>
<li>Familiarity with basic data filtering and sorting in SQL</li>
<li>Experience with simple aggregations (COUNT, SUM, AVG)</li>
</ul>
    
These prerequisites ensure participants have the foundational knowledge needed to succeed in learning the more advanced concepts like JOINs, subqueries, CTEs, and window functions covered in the second workshop.
</div>

## Table of Contents
1. [Introduction](#introduction)
2. [Relational Joins](#joins)
3. [Subqueries](#subqueries)
4. [Common Table Expressions (CTEs)](#ctes)
5. [Pivoting and Unpivoting](#pivot)
6. [Window Functions](#window)
7. [Key Points](#keypoints)

<div class="alert alert-success"> 
<b>Learning Goals</b><br><br>
By the end of this workshop you will be able to:
<ul>
<li>Combine data from multiple tables using different types of JOINs (INNER, LEFT, SELF).</li>
<li>Understand the role of primary and foreign keys in establishing table relationships.</li>
<li>Write and use subqueries to break down complex queries with multiple logical steps.</li>
<li>Simplify complex queries using Common Table Expressions (CTEs).</li>
<li>Transform data between row and column orientations with pivoting and unpivoting techniques.</li>
<li>Apply window functions to perform calculations across specified sets of rows.</li>
</ul>
</div>

In [16]:
# Here are some packages that we will need:

from IPython.display import SVG
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

<a id='introduction'></a>
## 1 · Introduction 

Estimated Time: 5 minutes

In the last workshop we covered many different operations one can perform in a given table. However, going back to the idea of Relational Databases, we often won't have all of the information we need on a single table, and we will need to find a way of cross-referencing the information from two (or more) different tables together. This processing is called "Joining", and it is an essential aspect of querying with SQL. 

Another common procedure is to first modify the data, and then perform some operations in the modified data. This is what Subqueries are used for - they allow us to quickly reference a modified version of the dataset we are querying, or compare information between two tables without joining them. Common Table Expressions are a way of doing many subqueries simultaneously, or very complex subqueries, while keeping things readable and organized.

Lastly, we will cover a class of operations called Window Functions, which are some of the most powerful tools in SQL. Instead of performing the same function to all rows, they allow the functions to only be applied to *windows*, which are a subset of the observations that are related in a prespecified way. This allows us to calculate summary statistics similar to what we did using ` GROUP BY `, but still retain the original disaggregated information.

📝 **Poll 1:** How confident do you feel about everything we covered in Part 1 (basic SELECT / JOIN / GROUP BY)?

<a id='joins'></a>
## 2 · Relational Joins 

Estimated Time: 35 minutes


**Purpose:** ```JOINs``` allow us to combine information from multiple different tables into one. When used with Querying, this also provides a way of retaining only the information required for a particular analysis into a single, organized table, even if originally this information was spread out  

**Example:** Say that we have two tables, one recording the information about Managers, and another one with the information about Employees. Each employee produces a certain amount of revenue to the firm, but a higher-up is interested in understanding what Manager had direct employees that produce the most revenue. 

If we had a table that listed each manager, their respective employees and how much they earned, we could use ```SUM``` + ```GROUP BY``` to quickly do this analysis. But we don't - the information is stored in two separate tables. This is exactly the context of `JOIN`s - creating a new table combining the information from two (or more) other tables. Here is a quick diagram to understand what is happening:

![Entity Relationship diagram linking Employees, Departments and Managers tables with Primary and Foreign Key arrows to show one-to-many relationships.](../Images/database-relationship-diagram2.svg)
---

Notice that in this example, employee_id would be the *primary key* of each table - a unique identifier for each row - while the column "report_to" on the Employees table serves as a *foreign key* - it serves as a reference to values on the table Managers which can be used to cross-reference information.

Notice also that we didn't keep all of the information from each table in the end result. By querying the table after joining, we can decide what to retain, and even perform operations or filtering directly! This again illustrates the principle we laid out at the beginning - SQL is a tool capable of preparing large amounts of data to be analyzed with libraries such as pandas.

In [17]:
# Using the unified dataset for JOIN examples
# First, ensure we have both customers and income tables loaded
print("Tables available in database:\n")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print([table[0] for table in cursor.fetchall()])

# Basic JOIN example - customers with their income
join_query = """
SELECT 
    c.name AS customer_name,
    c.city,
    c.items_purchased * c.price_per_item AS total_spent,
    i.income_source,
    i.amount AS income_amount
FROM 
    customers c
JOIN 
    income i ON c.name = i.name
WHERE 
    c.items_purchased IS NOT NULL 
    AND c.price_per_item IS NOT NULL
ORDER BY 
    c.name, i.income_source
LIMIT 10;
"""

print("\n JOIN Example - Customers with Income: \n")
result = pd.read_sql_query(join_query, conn)
print(result)

Tables available in database:

['customers', 'income', 'demo_customers', 'demo_orders']

 JOIN Example - Customers with Income: 

    customer_name       city  total_spent income_source  income_amount
0    Ahmed Hassan     Sydney       177.45    Consulting         1500.0
1    Ahmed Hassan     Sydney       177.45        Rental          800.0
2    Ahmed Hassan     Sydney       177.45        Salary         3500.0
3  Diego Martinez  Amsterdam        13.09        Salary         2700.0
4     Elena Popov     Madrid        56.37       Pension         1800.0
5   Lars Andersen       None       125.88        Salary         4500.0
6          Li Wei      Tokyo       155.98    Investment          200.0
7          Li Wei      Tokyo       155.98        Salary            NaN
8   Mary Williams       None      1183.71     Part-time         1200.0
9   Sarah Johnson     Berlin       301.15        Salary            NaN


Let's Break down exactly what is happening as we join tables using the example below::


```sql
SELECT 
    m.name AS "Manager Name",
    e.name AS "Employee Name",
    e.revenue AS "Revenue"
FROM 
    Managers m 
JOIN 
    Employees e 
ON 
    m.employee_id = e.reports_to
ORDER BY
    m.name, e.name
```

Let's start from the inside out:
- The first operation being performed is "FROM". This is telling us which Table we will be using as our "main" table - which in SQL is usually referred to as the Left Table. It is useful to think of any JOIN operation as acting "to the right" of this table.
    - Notice that we include a "m" right next to the name of the table we are importing - this is what we call an alias. While this is not necessary for a simple JOIN statement like this, it help keep the query clean and organized. And, as we will see later on, it is necessary for more advanced JOIN statements (such as self-joins).
- The second operation being performed is "JOIN". This is essentially telling SQL to consider a table that is a combination of the Left and Right tables.
    - One very important aspect is that we must tell SQL how to merge these two tables - which is what we do by using the ON statement.
    - In more advanced queries, ON statements can use very complex conditions, including compound ones using logical operators such as AND/OR/NOT, but for now we are just saying "I want a row that combines the information from the two tables whenever the column "employeed_id" on the Manager table matches the "reports_to" column on the Employees table.
    - Notice that we had to specify which table each column came from here - and we can already see why aliases can come in handy.
    - A very important thing to notice is that, if there are multiple matches to the condition on the ON statement, any rows that satisfy it will be joined. This is what happened in our example - there are two rows in the Merged table that have Bob as the Manager Name, since he had two employees that reported to him.
    - Another important thing to know is that, in a basic JOIN statement, any rows that go unmatched are not included in the final result. We will see how more advanced JOIN statements - such as LEFT JOIN - allow us to bypass this.
- We then go to our SELECT statement. As before, this is just telling SQL which information we actually want it to retrieve for us. But, since the information now might come from more than one source, we again need to specify from which table each column is coming from.
- The rest of the statements is similar to a basic Query - we can filter, order, limit/offset, etc - just like we did in the previous workshop


![Flow diagram with FROM, JOIN, WHERE, SELECT and ORDER BY blocks connected by arrows to show query processing order.](../Images/sql-query-flow.svg)
---

🏋️‍♂️ **Challenge:** Your First Real JOIN 

Individuall, the tables "customers" tells us who our clients are and "orders" tells us what they bought and when.

Task: Using a single JOIN, produce a table that answers
“Which orders were placed by which customer, in what country?”

```sql
-- Fill in the JOIN condition
SELECT
       o.order_id,
       o.order_date,
       c.name,
       c.country
FROM   orders    AS o
JOIN   customers AS c
       ON  _____________          
ORDER  BY o.order_date;
```


⚠️ **Warning:** Always make sure that your joining condition is present and accurate. Otherwise SQL will perform what we call a Cartesian Product - making every possible combination between the rows - which can become huge really fast.

In [18]:
# Example showing cartesian product vs correct join
# Using our existing customers and income tables

# Cartesian product (WRONG - no join condition)
cartesian_query = """
SELECT COUNT(*) as row_count
FROM customers, income;
"""
cartesian_result = pd.read_sql_query(cartesian_query, conn)

# Correct join
correct_join_query = """
SELECT COUNT(*) as row_count
FROM customers c
JOIN income i ON c.name = i.name;
"""
correct_result = pd.read_sql_query(correct_join_query, conn)

# Get actual counts
customer_count = pd.read_sql_query("SELECT COUNT(*) as count FROM customers", conn).iloc[0,0]
income_count = pd.read_sql_query("SELECT COUNT(*) as count FROM income", conn).iloc[0,0]

print("Cartesian Product Demonstration:\n")
print(f"Number of customers: {customer_count}")
print(f"Number of income records: {income_count}")
print(f"Cartesian product rows (WRONG): {cartesian_result.iloc[0,0]} ({customer_count} × {income_count} = every possible combination) \n")
print(f"Correct JOIN rows: {correct_result.iloc[0,0]} (only matching names)\n")

# Show a sample of what cartesian product looks like
cartesian_detail = """
SELECT c.name, c.city, i.income_source, i.amount
FROM customers c, income i
LIMIT 10;
"""
print(pd.read_sql_query(cartesian_detail, conn))

Cartesian Product Demonstration:

Number of customers: 25
Number of income records: 22
Cartesian product rows (WRONG): 550 (25 × 22 = every possible combination) 

Correct JOIN rows: 16 (only matching names)

         name      city income_source  amount
0  John Smith  New York        Salary  3000.0
1  John Smith  New York     Freelance   500.0
2  John Smith  New York        Salary  2500.0
3  John Smith  New York         Bonus  1000.0
4  John Smith  New York        Salary     NaN
5  John Smith  New York    Investment   200.0
6  John Smith  New York        Salary  3500.0
7  John Smith  New York    Consulting  1500.0
8  John Smith  New York        Rental   800.0
9  John Smith  New York        Salary     NaN


⚠️ **Warning:** Always verify your JOIN produces the expected number of rows! A missing or incorrect JOIN condition can create a Cartesian product, multiplying your data exponentially. For example, joining two 1,000-row tables without proper conditions creates 1,000,000 rows! Always check row counts after JOINs.

## 2.1 `ON` versus `USING`
While the ON clause is extremely flexible, if the two tables share a column, we can use the simpler USING() method.

⚠️  **Warning:** When using "USING" to JOIN tables, always make sure that the names are identical in both tables.

In [19]:
# USING clause with a subquery that creates matching column names
# Create a subquery that has a 'name' column matching customers table
using_join_query = """
SELECT 
    c.name,
    c.city,
    c.account_balance,
    i.total_income
FROM 
    customers c
JOIN 
    (SELECT name, SUM(amount) AS total_income 
     FROM income 
     GROUP BY name) i
USING (name)
WHERE 
    c.account_balance IS NOT NULL
ORDER BY 
    total_income DESC
LIMIT 10;
"""

print("JOIN with USING clause:\n")
print(pd.read_sql_query(using_join_query, conn))

# Compare with ON clause - same results, different syntax
on_join_query = """
SELECT 
    c.name,
    c.city,
    c.account_balance,
    i.total_income
FROM 
    customers c
JOIN 
    (SELECT name, SUM(amount) AS total_income 
     FROM income 
     GROUP BY name) i
ON c.name = i.name
WHERE 
    c.account_balance IS NOT NULL
ORDER BY 
    total_income DESC
LIMIT 10;
"""

print("\n Same query with ON clause (identical results):\n")
print(pd.read_sql_query(on_join_query, conn))

JOIN with USING clause:

             name       city  account_balance  total_income
0    Ahmed Hassan     Sydney           179.64        5800.0
1   Lars Andersen       None           588.43        4500.0
2      John Smith   New York           945.55        3500.0
3    Maria Garcia     London           905.34        3500.0
4     Yuki Tanaka   Shanghai           344.21        3200.0
5  Diego Martinez  Amsterdam           821.98        2700.0
6     Elena Popov     Madrid           845.86        1800.0
7   Mary Williams       None           795.02        1200.0
8          Li Wei      Tokyo           638.11         200.0

 Same query with ON clause (identical results):

             name       city  account_balance  total_income
0    Ahmed Hassan     Sydney           179.64        5800.0
1   Lars Andersen       None           588.43        4500.0
2      John Smith   New York           945.55        3500.0
3    Maria Garcia     London           905.34        3500.0
4     Yuki Tanaka   Shang

⚠️ Warning: Common Mistake - Ambiguous column names in JOINs

```sql
-- ❌ WRONG: Unclear which table's 'name' column
SELECT name, amount
FROM customers
JOIN income ON customers.name = income.name

-- ✅ CORRECT: Qualify columns with table names or aliases
SELECT c.name, i.amount
FROM customers c
JOIN income i ON c.name = i.name
```

But is important to be careful - using a non-primary key column can lead to issues

In [20]:
problematic_using_join = """
SELECT
    c.name,
    c.city,
    c.items_purchased * c.price_per_item AS total_spent,
    i.amount
FROM   customers AS c
INNER JOIN income AS i            -- multiple income rows per customer
USING  (name)                     -- alias *i* is required for i.amount
ORDER  BY c.name
LIMIT  20;
"""

print(
    "\nProblematic USING() JOIN Results:"
)
display(pd.read_sql_query(problematic_using_join, conn))


Problematic USING() JOIN Results:


,name,city,total_spent,amount
0,Ahmed Hassan,Sydney,177.45,800.0
1,Ahmed Hassan,Sydney,177.45,1500.0
2,Ahmed Hassan,Sydney,177.45,3500.0
3,Diego Martinez,Amsterdam,13.09,2700.0
4,Elena Popov,Madrid,56.37,1800.0
5,John Smith,New York,NaN,500.0
6,John Smith,New York,NaN,3000.0
7,Lars Andersen,None,125.88,4500.0
8,Li Wei,Tokyo,155.98,NaN
9,Li Wei,Tokyo,155.98,200.0


🔔 **Question:** Can anyone figure out what went wrong here?

🥊 **Challenge:** Join customers with income to list each customer’s name, country, and total income (sum of amount) sorted by total income descending.

In [15]:
# Debug this intentional error

bad_query = """
SELECT name, country, SUM(amount) AS total_income
FROM customers
JOIN income
  ON name = name          -- ❌ ambiguous columns!
GROUP BY name, country
ORDER BY total_income DESC;
"""
pd.read_sql_query(bad_query, conn)

DatabaseError: Execution failed on sql '
SELECT name, country, SUM(amount) AS total_income
FROM customers
JOIN income
  ON name = name          -- ❌ ambiguous columns!
GROUP BY name, country
ORDER BY total_income DESC;
': ambiguous column name: name

## 2.2 Multiple Joins - Advanced Topic, Time Permitting 🕤
We don't have to stop at two - we can use multiple joins at once. And what is interesting is that we can get pretty creative with the ON conditions.

In [21]:
multi_join_query = """
SELECT
    c.name,
    c.country,
    c.items_purchased * c.price_per_item AS total_spent,

    /* total income per customer, pre-aggregated in an inline view */
    it.total_income,

    /* country-level average balance for extra JOIN practice */
    cs.avg_balance

FROM   customers AS c

/* 1) Join to per-customer income totals */
LEFT JOIN (
    SELECT name,
           SUM(amount) AS total_income
    FROM   income
    GROUP  BY name
) AS it
  ON c.name = it.name

/* 2) Join to per-country balance stats */
LEFT JOIN (
    SELECT country,
           AVG(account_balance) AS avg_balance
    FROM   customers
    WHERE  account_balance IS NOT NULL
    GROUP  BY country
) AS cs
  ON c.country = cs.country

ORDER BY it.total_income DESC
"""

print("\nMulti-join demonstration – customers plus derived totals:\n")
print(pd.read_sql_query(multi_join_query, conn))



Multi-join demonstration – customers plus derived totals:

                name country  total_spent  total_income  avg_balance
0       Ahmed Hassan      AU       177.45        5800.0       179.64
1      Lars Andersen    None       125.88        4500.0          NaN
2         John Smith      US          NaN        3500.0       945.55
3       Maria Garcia      GB          NaN        3500.0       905.34
4        Yuki Tanaka      CN       299.36        3200.0       344.21
5     Diego Martinez      NL        13.09        2700.0       821.98
6        Elena Popov      ES        56.37        1800.0       845.86
7      Mary Williams    None      1183.71        1200.0          NaN
8      Sarah Johnson      DE       301.15         400.0          NaN
9             Li Wei      JP       155.98         200.0       638.11
10        Emma Brown      FR       517.44           NaN       929.69
11  Carlos Rodriguez      IN      1049.40           NaN       140.70
12     Anna Kowalski      BR      1066.01  

🔔 **Question:** In the results above, notice how some customers have NULL values for `total_income` while others have NULL for `total_spent`. Why is this happening?

This demonstrates the difference between ```INNER JOIN``` and ```LEFT JOIN```.

If we had used ```INNER JOIN``` for both joins, we would only see customers who have entries in ALL three tables. But the query above uses ```LEFT JOIN```, which preserves all rows from the left table (customers) even when there's no match in the joined tables.

Looking at our results:
- Emma Brown, Carlos Rodriguez, and others have `total_spent` values but NULL for `total_income` - they made purchases but have no income records
- Hans Schmidt and Fatima Al-Said have NULL for both - they're in the customers table but have neither purchase nor income data
- The first 10 customers have values for both - they exist in all three derived tables

This flexibility of ```LEFT JOIN``` is exactly why it's so useful - it helps us identify missing data relationships while still showing all our customers.

In this workshop we will explore both ```INNER JOIN``` (which we saw in the basic example) and ```LEFT JOIN``` in detail, along with ```SELF JOIN``` - but for those curious, there are others, such as ```RIGHT JOIN```, ```FULL JOIN``` AND ```CROSS JOIN```.

📝 **Poll 2:** You want *all* customers, even those with no orders. Which join keeps them? 
- customers `LEFT JOIN` orders
- orders `LEFT JOIN` customers
- Either order gives same result
- Neither command would return all customers

## 2.3 Advanced Join Variants
- **LEFT JOIN**: retains all rows from the left table.
    - This is why it is important to distinguish which table is being used in the FROM statement, and which is being brought by the JOIN statement
    - Specially useful to handle missing data 
- **SELF JOIN**: the table is joined to itself
    - We essentially deal with two tables - one of them being a duplicate of the first - and then JOIN them
    - Very useful as a filtering tool

🔔 **Question:** Can anyone think of an example in which we might want to join a table with itself?

![Set-logic Venn diagrams for LEFT, INNER and FULL joins with tiny code snippets and labelled A/B table overlaps.](../Images/sql-joins-venn-diagram.svg)
---

Let's understand what is going on with the different type of joins:
- In a ```LEFT JOIN```, we start with all rows from table 1 - the "left" table. If there is a match in table 2, bring the extra columns, otherwise leave these columns ```NULL```
- In a ```INNER JOIN```, we only keep the records that are on both tables! In other words, if one row is in one table but not the other, do not bring it to the merged table.
- For a ```FULL JOIN```, we treat both tables as the "left" one - we keep every row from both sides, and if it does not have a matching record, fill the gaps with ```NULL```

🙋 **Hands-Up:** True or False — `INNER JOIN` can drop rows that exist in only one table.

### 🏋️‍♂️ Challenge — Why `LEFT JOIN` Order Matters

`customers` lists everyone in our sample.  
`income` records dollar amounts **only for those customers who reported income**.

1. **Write two queries**  
   * **Query A**: `customers  LEFT JOIN income`  
   * **Query B**: `income     LEFT JOIN customers`
2. For each query return just one column: `COUNT(*) AS row_count`.
3. Compare the two row counts—explain the difference in one sentence *(code comment is fine)*.

> **Hint**  
> In a `LEFT JOIN`, **all** rows from the table **before** `LEFT JOIN` are preserved, even when no match exists in the table **after** `LEFT JOIN`.


In [22]:
qA = """SELECT COUNT(*) AS n FROM customers
        LEFT JOIN income ON customers.name = income.name"""
qB = """SELECT COUNT(*) AS n FROM income
        LEFT JOIN customers ON customers.name = income.name"""
nA = pd.read_sql_query(qA, conn)["n"][0]
nB = pd.read_sql_query(qB, conn)["n"][0]

assert nA >= nB, textwrap.dedent(f"""
    Expected Query A (customers left) to have ≥ rows than Query B,
    but got {nA} vs {nB}.
""")
print(f"✔ Query A rows: {nA}  |  Query B rows: {nB} — order matters!")

✔ Query A rows: 31  |  Query B rows: 22 — order matters!


One consideration when using ```LEFT JOIN``` is that it returns ```NULL``` values for unmatched records - but we might want to handle these NULLs differently depending on our analysis needs. 

For example, in our data:
- When Li Wei has a NULL amount in some income records, it means the income amount is unknown
- When Emma Brown has NULL for total_income after our JOIN, it means she has no income records at all

Sometimes we want to treat these NULLs as zeros (e.g., "no income records" = "0 total income" for a spending analysis), while other times we want to preserve them as unknown values.

Luckily, SQL includes a function that allows us to tailor the behavior of ```NULL``` entries for a given query - the ```COALESCE``` command. The first argument of the function tells us which column to analyze, and the second entry what to replace ```NULL``` values by. Let's take a look at an example in practice.

In [23]:
# LEFT JOIN without COALESCE – observe the NULL "amount" rows
no_coalesce_query = """
SELECT
    c.name,
    c.country,
    i.amount            -- will be NULL if the customer has no income rows
FROM   customers AS c
LEFT JOIN income AS i
       ON c.name = i.name
ORDER  BY c.name
LIMIT  20;
"""
print(pd.read_sql_query(no_coalesce_query, conn))


                name country  amount
0       Ahmed Hassan      AU   800.0
1       Ahmed Hassan      AU  1500.0
2       Ahmed Hassan      AU  3500.0
3        Aisha Patel      MX     NaN
4       Anna Ivanova      TH     NaN
5      Anna Kowalski      BR     NaN
6   Carlos Rodriguez      IN     NaN
7     Diego Martinez      NL  2700.0
8        Elena Popov      ES  1800.0
9         Emma Brown      FR     NaN
10    Fatima Al-Said      TR     NaN
11      Hans Schmidt      IT     NaN
12    Isabella Silva      HK     NaN
13       Ivan Petrov      SE     NaN
14      James Wilson      CA     NaN
15        John Smith      US   500.0
16        John Smith      US  3000.0
17          Jun Park      KR     NaN
18     Lars Andersen    None  4500.0
19            Li Wei      JP     NaN


In [24]:
# Same JOIN, but fill the missing income amounts with 0
coalesce_query = """
SELECT
    c.name,
    c.country,
    COALESCE(i.amount, 0) AS amount_received   -- replaces NULL with 0
FROM   customers AS c
LEFT JOIN income AS i
       ON c.name = i.name
ORDER  BY c.name
LIMIT  20;
"""
print(pd.read_sql_query(coalesce_query, conn))


                name country  amount_received
0       Ahmed Hassan      AU            800.0
1       Ahmed Hassan      AU           1500.0
2       Ahmed Hassan      AU           3500.0
3        Aisha Patel      MX              0.0
4       Anna Ivanova      TH              0.0
5      Anna Kowalski      BR              0.0
6   Carlos Rodriguez      IN              0.0
7     Diego Martinez      NL           2700.0
8        Elena Popov      ES           1800.0
9         Emma Brown      FR              0.0
10    Fatima Al-Said      TR              0.0
11      Hans Schmidt      IT              0.0
12    Isabella Silva      HK              0.0
13       Ivan Petrov      SE              0.0
14      James Wilson      CA              0.0
15        John Smith      US            500.0
16        John Smith      US           3000.0
17          Jun Park      KR              0.0
18     Lars Andersen    None           4500.0
19            Li Wei      JP              0.0


### SELF JOIN - Advanced Topic, Time Permitting 🕤

Now let's talk a bit about SELF JOIN. As we mentioned before, the idea here is to join a table with itself. This is one of the cases in which using aliases is extremely important, since, by construction, both of the tables will have all columns in common!

Let's see an example on how this might work in practice.

In [25]:
# A self-join example: customers with similar spending levels
spending_similarity_query = """
SELECT 
    c1.name AS customer_1,
    c2.name AS customer_2,
    c1.items_purchased * c1.price_per_item AS customer_1_spent,
    c2.items_purchased * c2.price_per_item AS customer_2_spent,
    ABS((c1.items_purchased * c1.price_per_item) - 
        (c2.items_purchased * c2.price_per_item)) AS spending_difference
FROM 
    customers c1
JOIN 
    customers c2 ON c1.name < c2.name
WHERE 
    c1.items_purchased IS NOT NULL 
    AND c1.price_per_item IS NOT NULL
    AND c2.items_purchased IS NOT NULL 
    AND c2.price_per_item IS NOT NULL
    AND ABS((c1.items_purchased * c1.price_per_item) - 
            (c2.items_purchased * c2.price_per_item)) < 50
ORDER BY 
    spending_difference;
"""

print("Customers with Similar Spending (within $50):\n")
print(pd.read_sql_query(spending_similarity_query, conn))

Customers with Similar Spending (within $50):

        customer_1        customer_2  customer_1_spent  customer_2_spent  \
0    Sarah Johnson       Yuki Tanaka            301.15            299.36   
1           Li Wei     Michel Dubois            155.98            143.16   
2    Anna Kowalski  Carlos Rodriguez           1066.01           1049.40   
3      Aisha Patel     Lars Andersen            109.14            125.88   
4    Lars Andersen     Michel Dubois            125.88            143.16   
5      Ivan Petrov         Raj Kumar            399.48            380.50   
6     Ahmed Hassan            Li Wei            177.45            155.98   
7     James Wilson     Sarah Johnson            331.04            301.15   
8    Lars Andersen            Li Wei            125.88            155.98   
9     James Wilson       Yuki Tanaka            331.04            299.36   
10     Aisha Patel     Michel Dubois            109.14            143.16   
11    Ahmed Hassan     Michel Dubois     

There are a few things worth noticing here:
- When we selected the columns from each table, it was important to rename them - since they had the same original names!
- Notice that John did not report to anyone - so he was not included as an employee in the merged table. This could be adapted by using a LEFT JOIN
- Some managers appear many times, since more than one employee reports to them.



### 🏋️‍♂️ Challenge — Customers Who Out-Spend Everyone Else in Their Country
“Who is the top spender in each country?”

Idea:
Self-JOIN customers to itself on country.
For each row c1, look for a matching row c2 in the same country whose total_spent is higher.
If no such c2 exists, c1 must be that country’s top spender.

In [26]:
top_spenders_query = """
-- 1) Build total spending per customer
WITH customer_spending AS (
    SELECT
           name,
           country,
           items_purchased * price_per_item AS total_spent
    FROM   customers
    WHERE  items_purchased IS NOT NULL
       AND price_per_item  IS NOT NULL
)

-- 2) Self-JOIN filter: keep only rows with no higher spender in the same country
SELECT
       c1.name,
       c1.country,
       c1.total_spent
FROM   customer_spending AS c1
LEFT JOIN customer_spending AS c2
       ON  c1.country      = c2.country
       AND c2.total_spent  > c1.total_spent
WHERE  c2.name IS NULL              -- ← nobody beats c1 in that country
ORDER  BY c1.country, c1.total_spent DESC;
"""
pd.read_sql_query(top_spenders_query, conn)



,name,country,total_spent
0,Mary Williams,None,1183.71
1,Lars Andersen,None,125.88
2,Sofia Santos,AE,595.32
3,Ahmed Hassan,AU,177.45
4,Anna Kowalski,BR,1066.01
5,James Wilson,CA,331.04
6,Yuki Tanaka,CN,299.36
7,Sarah Johnson,DE,301.15
8,Lucy Chen,EG,1102.08
9,Elena Popov,ES,56.37


Of course, SELF JOINs can be used for purposes other than filtering. Another classic application is to find pairs that satisfy some criterion

In [27]:
self_join_query = """
SELECT 
    c1.name AS customer_1,
    c2.name AS customer_2,
    c1.city AS shared_city,
    c1.country
FROM 
    customers c1
JOIN 
    customers c2 ON c1.city = c2.city 
                 AND c1.name < c2.name  -- Avoid duplicates and self-matches
WHERE 
    c1.city IS NOT NULL
ORDER BY 
    c1.city, c1.name;
"""

print("SELF JOIN Example - Customers from the Same City:")
result = pd.read_sql_query(self_join_query, conn)
print(result)

SELF JOIN Example - Customers from the Same City:
Empty DataFrame
Columns: [customer_1, customer_2, shared_city, country]
Index: []


📝 **Poll 3:** When self-joining `customers` to find same-country pairs, what extra condition avoids duplicate & mirror rows? - `c1.id <> c2.id` - `c1.id < c2.id` - `c1.country IS NOT NULL`

<a id='subqueries'></a>
## 3 · Subqueries 

Estimated Time: 20 minutes

**Definition.** As the name says, a subquery is a query contained in another query. This allows us to perform auxiliary queries, and then use the results of these queries in our main query. Unlike the main query, subqueries are temporary - they only exist while the instance of the query is being worked out. 

**Purpose.** Subqueries are very useful when one needs to break down a complex question into multiple manageable individual parts.For example, one might want to summarize or filter a given table, and use the summarized/filtered results as the input of another query. 

**Common Use Case.** Getting summary statistics for each individual, while keeping information unrelated from the variable we are using to aggregate.

In [28]:
# Total income per person, then attach all customer metadata
income_totals_query = """
SELECT
    c.name,
    c.city,
    c.country,
    it.total_income
FROM   customers AS c
JOIN  ( SELECT name,
               SUM(amount) AS total_income
        FROM   income
        WHERE  amount IS NOT NULL      -- defensive
        GROUP  BY name ) AS it
      ON c.name = it.name
ORDER BY it.total_income DESC;
"""
print("Customers ranked by total recorded income:")
display(pd.read_sql_query(income_totals_query, conn))


Customers ranked by total recorded income:


,name,city,country,total_income
0,Ahmed Hassan,Sydney,AU,5800.0
1,Lars Andersen,None,None,4500.0
2,John Smith,New York,US,3500.0
3,Maria Garcia,London,GB,3500.0
4,Yuki Tanaka,Shanghai,CN,3200.0
5,Diego Martinez,Amsterdam,NL,2700.0
6,Elena Popov,Madrid,ES,1800.0
7,Mary Williams,None,None,1200.0
8,Sarah Johnson,Berlin,DE,400.0
9,Li Wei,Tokyo,JP,200.0


There are two main ways of using subqueries:

1) To Filter results.

A classic example is to find all customers who spend more than average. 

In [29]:
# Subquery example - Find customers who spend more than average
subquery_example = """
WITH spending_stats AS (
    SELECT 
        AVG(items_purchased * price_per_item) AS avg_spent,
        MAX(items_purchased * price_per_item) AS max_spent,
        MIN(items_purchased * price_per_item) AS min_spent
    FROM customers
    WHERE items_purchased IS NOT NULL 
      AND price_per_item IS NOT NULL
)
SELECT 
    c.name,
    c.city,
    c.country,
    c.items_purchased * c.price_per_item AS total_spent,
    ROUND((SELECT avg_spent FROM spending_stats), 2) AS average_spending
FROM 
    customers c
WHERE 
    c.items_purchased IS NOT NULL 
    AND c.price_per_item IS NOT NULL
    AND (c.items_purchased * c.price_per_item) > (
        SELECT avg_spent FROM spending_stats
    )
ORDER BY 
    total_spent DESC;
"""

result = pd.read_sql_query(subquery_example, conn)
print("Customers who spend more than average:\n")
print(result)

avg_spent = pd.read_sql_query("""
    SELECT ROUND(AVG(items_purchased * price_per_item), 2) as avg_spent
    FROM customers
    WHERE items_purchased IS NOT NULL AND price_per_item IS NOT NULL
""", conn).iloc[0,0]
print(f"\n Average spending: ${avg_spent}")

Customers who spend more than average:

               name       city country  total_spent  average_spending
0          Jun Park      Seoul      KR      1847.94            557.12
1     Mary Williams       None    None      1183.71            557.12
2         Lucy Chen      Cairo      EG      1102.08            557.12
3     Anna Kowalski  São Paulo      BR      1066.01            557.12
4  Carlos Rodriguez     Mumbai      IN      1049.40            557.12
5      Anna Ivanova    Bangkok      TH       957.12            557.12
6    Isabella Silva  Hong Kong      HK       888.00            557.12
7      Sofia Santos      Dubai      AE       595.32            557.12

 Average spending: $557.12


⚠️ Warning: Common Mistake - Incorrect subquery references

```sql
-- ❌ WRONG: Cannot reference CTE columns directly
WITH high_spenders AS (
    SELECT name, SUM(amount) as total FROM income GROUP BY name
)
SELECT * FROM customers
WHERE account_balance > high_spenders.total

-- ✅ CORRECT: Use subquery or JOIN with CTE
WITH high_spenders AS (
    SELECT name, SUM(amount) as total FROM income GROUP BY name
)
SELECT c.* FROM customers c
JOIN high_spenders h ON c.name = h.name
WHERE c.account_balance > h.total
```

2) As a derived table to query from:

In [30]:
window_query_sqlite = """
SELECT
    name,
    city,
    items_purchased,
    city_avg
FROM (
    SELECT
        name,
        city,
        items_purchased,
        AVG(items_purchased) OVER (PARTITION BY city) AS city_avg
    FROM   customers
    WHERE  items_purchased IS NOT NULL
)
WHERE  items_purchased > city_avg
ORDER  BY city, items_purchased DESC;
"""

display(pd.read_sql_query(window_query_sqlite, conn))


,name,city,items_purchased,city_avg
0,Mary Williams,None,17.0,11.5


As promised in the first workshop, we can also use subqueries in combination with IN to check for membership against entire tables - usually the result of a subquery.

For example, let's say that we want to select the names of all customers who have purchased on Electronic item, but the information between consumers, products and purchases are all on separate tables. We could first JOIN them, then use a WHERE statement to filter them. But this would create a very large merged table. Instead, we can just use an IN statement with a subquery:

In [31]:
# Using IN with subquery to find customers with high income
# First, let's see what income levels we have
income_summary = """
SELECT 
    name,
    SUM(amount) as total_income
FROM income
WHERE amount IS NOT NULL
GROUP BY name
HAVING SUM(amount) > 5000
ORDER BY total_income DESC;
"""
print("High income earners (>$5000):\n")
print(pd.read_sql_query(income_summary, conn))

# Find customer details for high income earners
high_income_customers_query = """
SELECT 
    c.name,
    c.city,
    c.country,
    c.account_balance,
    c.items_purchased * c.price_per_item as total_spent
FROM 
    customers c
WHERE 
    c.name IN (
        SELECT name
        FROM income
        WHERE amount IS NOT NULL
        GROUP BY name
        HAVING SUM(amount) > 5000
    )
    AND c.account_balance IS NOT NULL
ORDER BY 
    c.account_balance DESC;
"""

print("\n Customer details for high income earners: \n")
result = pd.read_sql_query(high_income_customers_query, conn)
print(result)

# Compare with NOT IN - customers without high income
low_income_customers_query = """
SELECT 
    c.name,
    c.city,
    c.account_balance
FROM 
    customers c
WHERE 
    c.name NOT IN (
        SELECT name
        FROM income
        WHERE amount IS NOT NULL
        GROUP BY name
        HAVING SUM(amount) > 5000
    )
    AND c.name IN (SELECT DISTINCT name FROM income)  -- Only those with some income data
    AND c.account_balance IS NOT NULL
ORDER BY 
    c.account_balance DESC
LIMIT 5;
"""

print("\n Customers with income data but earning <= $5000:\n")
print(pd.read_sql_query(low_income_customers_query, conn))

High income earners (>$5000):

           name  total_income
0  Ahmed Hassan        5800.0

 Customer details for high income earners: 

           name    city country  account_balance  total_spent
0  Ahmed Hassan  Sydney      AU           179.64       177.45

 Customers with income data but earning <= $5000:

             name       city  account_balance
0      John Smith   New York           945.55
1    Maria Garcia     London           905.34
2     Elena Popov     Madrid           845.86
3  Diego Martinez  Amsterdam           821.98
4   Mary Williams       None           795.02


⚠️ **Warning:** Using NOT IN with subqueries that may contain NULL values can return no results! This is because NULL comparisons are undefined. Use NOT EXISTS or filter out NULLs in your subquery with WHERE column IS NOT NULL to avoid this trap.

🥊 **Challenge:** List the name, country, and total_income of every customer whose total income is below the overall average customer income.

In [16]:
# Debug this intentional error

bad_query = """
SELECT c.name, c.country, SUM(i.amount) AS total_income
FROM customers AS c
JOIN income AS i
  ON c.name = i.name
GROUP BY c.name, c.country
WHERE SUM(i.amount) <
      ( SELECT AVG(SUM(amount)) FROM income );  -- ❌ aggregate inside AVG; also WHERE not HAVING
"""
pd.read_sql_query(bad_query, conn)

DatabaseError: Execution failed on sql '
SELECT c.name, c.country, SUM(i.amount) AS total_income
FROM customers AS c
JOIN income AS i
  ON c.name = i.name
GROUP BY c.name, c.country
WHERE SUM(i.amount) <
      ( SELECT AVG(SUM(amount)) FROM income );  -- ❌ aggregate inside AVG; also WHERE not HAVING
': near "WHERE": syntax error

<a id='ctes'></a>
## 4 · Common Table Expressions (CTEs) 

Estimated Time: 15 minutes

Subqueries are a great way of breaking down a complex query into smaller, more manageable subparts. But very often these subqueries can become quite long, and given that one must include the full query inside another query, they can become very hard to read. 

CTEs are a way of solving this issue. Instead of rewriting the entire subquery inside the main query, we first give aliases to our subqueries, and then refer to them in the main query. 

![Infographic “CTEs vs Subqueries” showing when to use a readable CTE versus an inline subquery, with tiny code examples.](../Images/ctesubquery.svg) 


Basic Syntax   

```sql

WITH cte_name AS (
    SELECT column1, column2, ...
    FROM table
    WHERE condition)
SELECT * 
FROM cte_name;
Key Components
```

WITH clause: Introduces one or more CTEs
cte_name: Gives a name to the temporary result set
Main query: References the CTE like a regular table

🔔 **Question:** Would you say that CTEs improve readability? Can you think of an example in which the code is easier to read using subqueries instead?

Multiple CTEs

```sql

WITH cte1 AS (
    SELECT column1 FROM table1
),
cte2 AS (
    SELECT column2 FROM table2
)
SELECT *
FROM cte1
JOIN cte2 ON cte1.column = cte2.column;
```

💡 **Tip:** CTE's can be referenced multiple times in the same query, which improve readability and prevents errors!

🙋 **Hands-Up:** Which style feels clearer for multi-step queries so far? A. A nested subquery B. A CTE (`WITH …`) above the main query

In [32]:
# CTE example using customers and income data
cte_query = """
WITH customer_spending AS (
    -- Calculate total spending per customer
    SELECT 
        name,
        items_purchased * price_per_item AS total_spent,
        city,
        country
    FROM customers
    WHERE items_purchased IS NOT NULL 
      AND price_per_item IS NOT NULL
),
customer_income AS (
    -- Calculate total income per customer
    SELECT 
        name,
        SUM(amount) AS total_income
    FROM income
    WHERE amount IS NOT NULL
    GROUP BY name
),
spending_analysis AS (
    -- Combine spending and income data
    SELECT 
        cs.name,
        cs.city,
        cs.country,
        cs.total_spent,
        ci.total_income,
        ROUND(cs.total_spent * 100.0 / ci.total_income, 2) AS spending_rate
    FROM customer_spending cs
    JOIN customer_income ci ON cs.name = ci.name
    WHERE ci.total_income > 0
)
-- Final query using the CTEs
SELECT 
    name,
    city,
    country,
    total_spent,
    total_income,
    spending_rate,
    CASE 
        WHEN spending_rate > 50 THEN 'High Spender'
        WHEN spending_rate > 20 THEN 'Moderate Spender'
        ELSE 'Low Spender'
    END AS spending_category
FROM spending_analysis
ORDER BY spending_rate DESC;
"""

⚠️ **Warning:** We cannot reference CTEs directly in a query - this is what we call (lack of) "portability".

```sql
WITH average_data AS (
    SELECT AVG(value) AS avg_value FROM table
)
SELECT *
FROM other_table
WHERE value > average_data.avg_value 
```
Whenever we want to reference a CTE, we must either use a subquery:

```sql
WHERE value > (SELECT avg_value FROM average_data)
```

Or JOIN with the CTE:

```sql
JOIN average_data ON 1=1
WHERE value > average_data.avg_value
```

💡 **Tip:** In this last example, we used a common trick: choose an expression that is always true to add a constant column to the table.

🙋 **Hands-Up:** Which keyword starts a Common Table Expression? A. `WITH` B. `WHERE` C. `AS`

<a id='pivot'></a>
## 5 · Pivoting and Unpivoting (Or Melting) 

Estimated Time: 15 minutes

Pivoting is the process of turning rows into columns, Unpivoting (also called Melting) is the inverse process. 

A common application is when we would like to group information that applies to the same individual. For example, we might have a list of all the different transactions that different clients made, including amount and date. But maybe we would like to understand how purchases vary across the days of the week for each given customer. So we can turn a table from having many rows and three columns, to having one row for each consumer, and 8 columns: one representing the customer name (or any other identification), and one with the transaction amount for that customer on each day of the week. 

📝 **Poll 4:** In data-reshaping lingo, which operation produces a **tall (long-format)** table, and which yields a **wide (spread)** table?

- Melt (gather) → tall | Pivot (spread) → wide  
- Melt (gather) → wide | Pivot (spread) → tall  
- Both operations create tall tables  
- Both operations create wide tables


![Side-by-side pivot vs melt diagram converting wide daily-sales columns into tall name-date-sales rows and back again.](../Images/pivot-melt-diagram.svg)

This is done by using the ```UNION``` (or `UNION ALL`) functions - which are analogues to `JOIN`s, but instead of putting different columns side by side, they combine results from different queries on top of each other. 

The main difference between `UNION` and `UNION ALL` is that the former removes duplicate rows, which increases the computational cost of the function.

Two additional observations:
- The column names will be taken from the first one selected
- All `SELECT` statements must have the same number of columns, and corresponding columns must have the same data types.

In [33]:
# Using the income table for pivot example
print("Original Income Data (first 10 rows):")
income_sample = pd.read_sql_query("SELECT * FROM income LIMIT 10", conn)
print(income_sample)

# Pivot income by day of week
pivot_query = """
SELECT 
    name,
    SUM(CASE WHEN day = 'Monday' THEN amount ELSE 0 END) AS Monday,
    SUM(CASE WHEN day = 'Tuesday' THEN amount ELSE 0 END) AS Tuesday,
    SUM(CASE WHEN day = 'Wednesday' THEN amount ELSE 0 END) AS Wednesday,
    SUM(CASE WHEN day = 'Thursday' THEN amount ELSE 0 END) AS Thursday,
    SUM(CASE WHEN day = 'Friday' THEN amount ELSE 0 END) AS Friday,
    SUM(CASE WHEN day = 'Saturday' THEN amount ELSE 0 END) AS Saturday,
    SUM(amount) AS total_weekly_income
FROM income
WHERE amount IS NOT NULL
GROUP BY name
HAVING total_weekly_income > 0
ORDER BY total_weekly_income DESC;
"""

result = pd.read_sql_query(pivot_query, conn)
print("\\nPivoted Income Data (by day of week):")
print(result)

Original Income Data (first 10 rows):
            name income_source  amount        day
0     John Smith        Salary  3000.0     Monday
1     John Smith     Freelance   500.0     Friday
2   Maria Garcia        Salary  2500.0     Monday
3   Maria Garcia         Bonus  1000.0     Friday
4         Li Wei        Salary     NaN     Monday
5         Li Wei    Investment   200.0  Wednesday
6   Ahmed Hassan        Salary  3500.0     Monday
7   Ahmed Hassan    Consulting  1500.0    Tuesday
8   Ahmed Hassan        Rental   800.0   Thursday
9  Sarah Johnson        Salary     NaN     Monday
\nPivoted Income Data (by day of week):
               name  Monday  Tuesday  Wednesday  Thursday  Friday  Saturday  \
0      Ahmed Hassan  3500.0   1500.0        0.0     800.0     0.0       0.0   
1     Lars Andersen  4500.0      0.0        0.0       0.0     0.0       0.0   
2    Patricia Davis     0.0      0.0     4000.0       0.0     0.0       0.0   
3     Robert Taylor  2900.0      0.0        0.0       0.

Melting, the opposite process, is very useful when we want to do analysis regarding a variable that is not the one determining the rows. For example, say that we have sales data in which each row corresponds to a different product, and different columns represent different years. If we instead want to analyze the expenditure in different years, we can melt the table, and then use grouping or filtering to select a given year.

In [45]:
# First, let's create a pivoted view of income by source
pivoted_income = """
SELECT 
    name,
    SUM(CASE WHEN income_source = 'Salary' THEN amount ELSE 0 END) AS salary_income,
    SUM(CASE WHEN income_source = 'Freelance' THEN amount ELSE 0 END) AS freelance_income,
    SUM(CASE WHEN income_source = 'Investment' THEN amount ELSE 0 END) AS investment_income,
    SUM(CASE WHEN income_source = 'Consulting' THEN amount ELSE 0 END) AS consulting_income
FROM income
WHERE amount IS NOT NULL
GROUP BY name
HAVING (salary_income + freelance_income + investment_income + consulting_income) > 0
LIMIT 5;
"""

print("Pivoted Income Data (Wide Format):\n")
pivoted_df = pd.read_sql_query(pivoted_income, conn)
print(pivoted_df)

# Now melt it back to long format using UNION ALL
melt_query = """
WITH pivoted_data AS (
    SELECT 
        name,
        SUM(CASE WHEN income_source = 'Salary'    THEN amount ELSE 0 END) AS salary_income,
        SUM(CASE WHEN income_source = 'Freelance' THEN amount ELSE 0 END) AS freelance_income,
        SUM(CASE WHEN income_source = 'Investment' THEN amount ELSE 0 END) AS investment_income,
        SUM(CASE WHEN income_source = 'Consulting' THEN amount ELSE 0 END) AS consulting_income
    FROM income
    WHERE amount IS NOT NULL
    GROUP BY name
),
melted AS (
    SELECT name, 'Salary'    AS income_type, salary_income    AS amount FROM pivoted_data WHERE salary_income    > 0
    UNION ALL
    SELECT name, 'Freelance' AS income_type, freelance_income AS amount FROM pivoted_data WHERE freelance_income > 0
    UNION ALL
    SELECT name, 'Investment' AS income_type, investment_income AS amount FROM pivoted_data WHERE investment_income > 0
    UNION ALL
    SELECT name, 'Consulting' AS income_type, consulting_income AS amount FROM pivoted_data WHERE consulting_income > 0
)
SELECT *
FROM   melted
ORDER  BY name, income_type
LIMIT 10;
"""

print("\n Melted Income Data (Long Format):\n")
melted_df = pd.read_sql_query(melt_query, conn)
print(melted_df)

print("\n This demonstrates how UNION ALL can transform wide data back to long format!\n")

Pivoted Income Data (Wide Format):

              name  salary_income  freelance_income  investment_income  \
0     Ahmed Hassan         3500.0               0.0                  0   
1       Amanda Lee            0.0            2200.0                  0   
2   Diego Martinez         2700.0               0.0                  0   
3  Jennifer Wilson         3100.0               0.0                  0   
4       John Smith         3000.0             500.0                  0   

   consulting_income  
0             1500.0  
1                0.0  
2                0.0  
3                0.0  
4                0.0  

 Melted Income Data (Long Format):

              name income_type  amount
0     Ahmed Hassan  Consulting  1500.0
1     Ahmed Hassan      Salary  3500.0
2       Amanda Lee   Freelance  2200.0
3   Diego Martinez      Salary  2700.0
4  Jennifer Wilson      Salary  3100.0
5       John Smith   Freelance   500.0
6       John Smith      Salary  3000.0
7    Lars Andersen      Salary  

<a id='window'></a>
## 6 · Window Functions 

Estimated Time: 25 minutes

**Definition.** A window is a subset of the table that is related in some prespecified way, in a very similar way that `GROUP BY` operates - with the crucial difference that it does not collapse the different rows into one, which allows us to preserve information. 

Window Functions operate separately into different windows. Common examples are

- `RANK` - allow us to obtain the rank for different observations inside a given group. For example, we might want to see how much a consumer spent on his first purchase. So we would create a window for each consumer, apply the RANK window function, and then select those that have rank = 1.
- `DENSE_RANK` - Similar to rank, but RANK jumps ranks if there are ties, while DENSE_RANK always has rankings as consecutive numbers. You might have seen this happening in college rankings!
- `ROW_NUMBER()` - returns the row number of each observation. Very useful when the table lacks a primary key.
- `SUM` - when used as a window function, allow us to perform cumulative sums - for example, cumulative sales up to a given date by each salesperson.
- `AVG`/`MAX`/`MIN` - same as their aggregate versions, but allowing to keep information rather than collapsing rows.
- `LAG`/`LEAD` - very useful in the context of time series, allow us to look at the previous/next value of a series.

Basic Syntax: `WINDOW_FUNCTION() OVER (PARTITION BY columns ORDER BY columns)`

Exception: `SUM(column) OVER (PARTITION BY columns)`

🙋 **Hands-Up:** Is the difference between a *window function* and an *aggregate function* clear? A. Yes B. Not yet

![Table annotated with running totals and row numbers to visualize how SUM() OVER and RANK window functions scan partitions.
](../Images/window-function-visualization.svg)
---

### Example 1: ```ROW_NUMBER``` - Rank customers by account balance within each country

Unfortunately, sometimes we work with tables that do not have a primary key. A classic example is one listing many different transactions, including information such as customer, store, amount, method of payment - but no order id. ```ROW_NUMBER``` is a useful window function in this scenario, as it creates an additional column with, as the name suggests, the numbers of each column in the particular ordering the table is in. This works as a "fake" primary key that we can still use to perform operations such as ```JOIN```, even without a true primary key.

⚠️ **Warning:** Forgetting the PARTITION BY clause in window functions will calculate across the entire table, not within groups. This is a common mistake that can lead to incorrect business logic. Always double-check that your window function includes the appropriate PARTITION BY when you need group-wise calculations.

In [35]:

row_number_query = """
SELECT 
    name,
    country,
    account_balance,
    ROW_NUMBER() OVER (PARTITION BY country ORDER BY account_balance DESC) as balance_rank_in_country
FROM customers
WHERE country IS NOT NULL 
  AND account_balance IS NOT NULL
ORDER BY country, balance_rank_in_country
LIMIT 15;
"""
print("1. ROW_NUMBER Example - Customer Rankings by Country:")
print(pd.read_sql_query(row_number_query, conn))

1. ROW_NUMBER Example - Customer Rankings by Country:
                name country  account_balance  balance_rank_in_country
0       Sofia Santos      AE           352.84                        1
1       Ahmed Hassan      AU           179.64                        1
2      Anna Kowalski      BR           392.80                        1
3       James Wilson      CA           449.81                        1
4        Yuki Tanaka      CN           344.21                        1
5          Lucy Chen      EG           167.10                        1
6        Elena Popov      ES           845.86                        1
7         Emma Brown      FR           929.69                        1
8       Maria Garcia      GB           905.34                        1
9     Isabella Silva      HK           833.92                        1
10  Carlos Rodriguez      IN           140.70                        1
11      Hans Schmidt      IT           104.97                        1
12            Li Wei   

⚠️ Warning: Common Mistake - Missing PARTITION BY in window functions

```sql
-- ❌ WRONG: Ranks ALL rows together, not by group
SELECT name, country, balance,
       RANK() OVER (ORDER BY balance DESC) as rank
FROM customers

-- ✅ CORRECT: Use PARTITION BY to rank within groups
SELECT name, country, balance,
       RANK() OVER (PARTITION BY country ORDER BY balance DESC) as rank
FROM customers
```

### Example 2: Running total - Cumulative spending by purchase date

A very common application in business is to understand not just the spending or revenue in a given day, but the total amount spent/received up to a given date. This is the idea of a cumulative sum - the running total including previous purchases. In SQL, we can use the Window Function versiom of ```SUM``` - which is just like its aggregate function counterpart, but includes the characteristic OVER () component of window functions. This can be used to indicate both how we want to partition the dataset, and how we would like to order the subgroups, since the order matters when calculating cumulative sums!

In [36]:

running_total_query = """
SELECT 
    name,
    last_purchase,
    items_purchased * price_per_item as purchase_amount,
    SUM(items_purchased * price_per_item) 
        OVER (ORDER BY last_purchase ROWS UNBOUNDED PRECEDING) as running_total
FROM customers
WHERE last_purchase IS NOT NULL 
  AND items_purchased IS NOT NULL 
  AND price_per_item IS NOT NULL
ORDER BY last_purchase
LIMIT 10;
"""
print("\n2. Running Total Example - Cumulative Spending Over Time:")
print(pd.read_sql_query(running_total_query, conn))


2. Running Total Example - Cumulative Spending Over Time:
            name last_purchase  purchase_amount  running_total
0     Emma Brown    2024-01-28           517.44         517.44
1   James Wilson    2024-02-02           331.04         848.48
2    Ivan Petrov    2024-02-04           399.48        1247.96
3         Li Wei    2024-02-10           155.98        1403.94
4    Yuki Tanaka    2024-02-17           299.36        1703.30
5    Elena Popov    2024-03-02            56.37        1759.67
6  Lars Andersen    2024-04-08           125.88        1885.55
7      Raj Kumar    2024-04-10           380.50        2266.05
8   Ahmed Hassan    2024-05-14           177.45        2443.50
9    Aisha Patel    2024-06-20           109.14        2552.64


### Example 3: AVG with PARTITION BY - Compare to group average

One of the main motivations for the use of window functions is precisely to have an alternative to ```GROUP BY``` that still allow us to retain individual level information. This is very useful when trying to compare each individual element to a group it belongs - for example, compare the customer balance with the average balance for customers in his country. Without window functions, we would need to first aggregate the data using an aggregate function with ```GROUP BY```, and then use ```JOIN``` to make a new table, and then perform the comparison. Window functions allow us to do this directly.

In [37]:
avg_comparison_query = """
SELECT 
    name,
    country,
    account_balance,
    ROUND(AVG(account_balance) OVER (PARTITION BY country), 2) as country_avg_balance,
    ROUND(account_balance - AVG(account_balance) OVER (PARTITION BY country), 2) as diff_from_country_avg
FROM customers
WHERE country IS NOT NULL 
  AND account_balance IS NOT NULL
ORDER BY country, diff_from_country_avg DESC
LIMIT 10;
"""
print("\n 3. Window AVG Example - Customer Balance vs Country Average:")
pd.read_sql_query(avg_comparison_query, conn)


 3. Window AVG Example - Customer Balance vs Country Average:


,name,country,account_balance,country_avg_balance,diff_from_country_avg
0,Sofia Santos,AE,352.84,352.84,0.0
1,Ahmed Hassan,AU,179.64,179.64,0.0
2,Anna Kowalski,BR,392.80,392.80,0.0
3,James Wilson,CA,449.81,449.81,0.0
4,Yuki Tanaka,CN,344.21,344.21,0.0
5,Lucy Chen,EG,167.10,167.10,0.0
6,Elena Popov,ES,845.86,845.86,0.0
7,Emma Brown,FR,929.69,929.69,0.0
8,Maria Garcia,GB,905.34,905.34,0.0
9,Isabella Silva,HK,833.92,833.92,0.0


### Example 4: RANK vs DENSE_RANK - Ranking with ties

Another common example is to obtain the relative rank of observations in a given group. This is an incredibly complex task without window function - one would need to find all the possible groups, use ```WHERE``` commands to isolate them, order them, and then merge all of the isolated groups together. ```RANK``` allows us to perform this operation in a single command. 

One interesting feature of ```RANK``` is that it leaves gaps after ties. This is common procedure in some applications - for example college rankings. However we might want to not allow for these gaps, and just allow for multiple entries at the same rank. This is exactly what ```DENSE_RANK``` does - dense here referring to the fact that there are no gaps in the ranking.

In [38]:

rank_comparison_query = """
WITH income_totals AS (
    SELECT 
        name,
        SUM(amount) as total_income
    FROM income
    WHERE amount IS NOT NULL
    GROUP BY name
)
SELECT 
    name,
    total_income,
    RANK() OVER (ORDER BY total_income DESC) as rank_with_gaps,
    DENSE_RANK() OVER (ORDER BY total_income DESC) as dense_rank_no_gaps
FROM income_totals
ORDER BY total_income DESC
LIMIT 10;
"""
print("\n 4. RANK vs DENSE_RANK Example:")
print(pd.read_sql_query(rank_comparison_query, conn))


 4. RANK vs DENSE_RANK Example:
              name  total_income  rank_with_gaps  dense_rank_no_gaps
0     Ahmed Hassan        5800.0               1                   1
1    Lars Andersen        4500.0               2                   2
2   Patricia Davis        4000.0               3                   3
3       John Smith        3500.0               4                   4
4     Maria Garcia        3500.0               4                   4
5    Robert Taylor        3500.0               4                   4
6    Michael Brown        3400.0               7                   5
7      Yuki Tanaka        3200.0               8                   6
8  Jennifer Wilson        3100.0               9                   7
9   Diego Martinez        2700.0              10                   8


🏋️‍♂️ Challenge — Customers Who Out-Spend Their Country Average
(Window-Function Edition)

In the previous challenge you solved this task with the standard method:
GROUP BY → sub-query → JOIN.

Now let’s do the same calculation with a single window function.

Goal: List every customer whose total spending ( items_purchased × price_per_item ) is strictly higher than the average spending of customers in the same country.

In [39]:
country_avg_window_sql = """
WITH spending AS (
    SELECT
           name,
           country,
           items_purchased * price_per_item AS total_spent,
           AVG(items_purchased * price_per_item)
                 OVER (PARTITION BY country) AS country_avg
    FROM   customers
    WHERE  items_purchased IS NOT NULL
      AND  price_per_item  IS NOT NULL
)
SELECT *
FROM   spending
WHERE  total_spent > country_avg
ORDER  BY country, total_spent DESC;
"""

df = pd.read_sql_query(country_avg_window_sql, conn)
df

,name,country,total_spent,country_avg
0,Mary Williams,None,1183.71,654.795


<div class="alert alert-success">  
    
## Key Points

Estimated Time: 5 minutes

* **JOINs combine tables on matching values** - Use ON or USING to specify the relationship between tables
* **LEFT JOIN preserves all rows from the left table** - Even when there's no match in the right table, making it ideal for finding missing relationships
* **CTEs improve readability** - Use WITH clauses to break complex queries into named, reusable components
* **Window functions maintain row-level detail** - Unlike GROUP BY, they calculate aggregates while keeping all original rows
* **PARTITION BY creates calculation groups** - Similar to GROUP BY but for window functions
* **Subqueries can filter or transform data** - Use them in WHERE clauses for filtering or FROM clauses as derived tables
* **COALESCE handles NULL values gracefully** - Replace NULLs with meaningful defaults in JOINs and calculations
* **Self-JOINs compare rows within the same table** - Essential for finding relationships between records in a single table

</div>